# Coherence analysis for OAS database

## full paired dataset, human only

total sequences in paired dataset: 1'946'101 (no clustering but full OAS db)

In [1]:
# used julia version 1.10.2
using CSV
using DataFrames
using StatsBase


# Patient Information (Subjects)

In [2]:
file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/full_extraction_for_coherence_paired_data_extra_cols_header.csv" 
subjects_df = CSV.read(file_path, DataFrame)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String,String,String7,String15,String31,String15
1,GATTTCCTTAAATTCAGGGTCCAGCTCACATGGGAAATACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV4-59*01,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAG,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXYSSXXXXXXYFDYWGQGTLVTVSS,ARDVGPYNSISPGRYYFDY,GGGGTCACAAGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTGCTCCTCACCCTCCTCACTCAGGACACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAGGTCAGCCCAAGGCCAACCCCACTGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTCCAAGCCAACAAGGCCACACTAGTGTGTCTGATCAGTGACTTCTACCCGGGAGCTGTGACAGTGGCCTGGAAGGCAGATGGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCAAACCCTCCAAACAGAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV2-23*02,CAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAG,QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,QSALTQPASVSGSPGQSITISCTGTSSDVGSYNLVSWYQQHPGKAPKLMIYEVSKRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSXXXYVFGTGTKVTVL,CSYAGSRILYV,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,Unsorted-B-Cells,None,human,390c,"James et al, 2020",65 to 70
2,ATACTTTCTGAGAGCCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGAGGGGTCGTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCATCTGCAATGTCACTGGTTTCTCCATCAGTGGTTACTTCTGGAGCTGGGTCCGGCAGCCCCCAGGGAAGGGACTGGAATGGATTGCCTCTATGACTTACAGTGGCACCACCAACTACAACCCCTCCCTCCAGGGCCGAGTCACTATGTCACTTTCCATGTCCAAGGACCAGGTCTCCCTGAAACTGAGCTCTGCGACCGCTGCGGACACGGCCGTCTATTACTGTGCGAGAATGGCGCGAGATGGCTACGTTTTGCGGGACTGGTACTTCGATCTCTGGGGCCAAGGCACCCTGGTCACTGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCC

In [3]:
subjects_counts = countmap(subjects_df.Subject)
btype_counts = countmap(subjects_df.BType)
print(btype_counts)

Dict("Plasma-B-Cells" => 41614, "Memory-B-Cells" => 509879, "Unsorted-B-Cells" => 662832, "Naive-B-Cells" => 666907, "Plasmablast" => 9248, "ASC" => 1534, "CD27-memory-and-Plasmablast/Plasma-B-Cells" => 31232, "Plasmablast/Plasma-B-Cells" => 17257, "double-nagative-B-cells" => 4826, "RV+B-Cells" => 771)

### Preprocess the data 

In [4]:
# Filter out rows where BType is "Unsorted-B-Cells"
filtered_df_subjects = filter(row -> row.BType != "Unsorted-B-Cells", subjects_df);

# Extract up to the first two segments (e.g., "IGKV2-30" from "IGKV2-30*01")
filtered_df_subjects[!, :general_v_gene_heavy] = replace.(filtered_df_subjects.v_call_heavy, r"(^[^*]+?)(?:\*.*)?$" => s"\1");
filtered_df_subjects[!, :general_v_gene_light] = replace.(filtered_df_subjects.v_call_light, r"(^[^*]+?)(?:\*.*)?$" => s"\1");

# extract names of the BTypes
replace!(filtered_df_subjects.BType, "CD27-memory-and-Plasmablast/Plasma-B-Cells" => "CD27-memory-and-Plasmablast_Plasma-B-Cells")
replace!(filtered_df_subjects.BType, "Plasmablast/Plasma-B-Cells" => "Plasmablast_Plasma-B-Cells")

unique_btypes = unique(filtered_df_subjects.BType)

println("unique BTypes: ", unique_btypes)

# Group by BType
grouped_by_btype = groupby(filtered_df_subjects, :BType)

output_dir = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/OAS_data_grouped_by_BTypes/extra_cols"

# Save each group as a separate CSV file
for (btype_group_id, btype_group) in enumerate(grouped_by_btype)
    # Extract the BType name (to use in the file name)
    btype_name = btype_group.BType[1]  # Assuming BType exists and is consistent within each group

    # print number of rows in each group
    println("Number of rows in group $btype_name: ", nrow(btype_group))
    
    # Create a sanitized file name (replace spaces or special characters if needed)
    file_name = "BType_$(replace(btype_name, r"\s" => "_"))_extra_cols.csv"
    
    # Write the group to a CSV file
    CSV.write(joinpath(output_dir, file_name), btype_group, writeheader=true)
end

println("Grouped data saved to CSV files.")

unique BTypes: ["Memory-B-Cells", "Plasma-B-Cells", "Naive-B-Cells", "RV+B-Cells", "double-nagative-B-cells", "Plasmablast", "CD27-memory-and-Plasmablast_Plasma-B-Cells", "ASC", "Plasmablast_Plasma-B-Cells"]
Number of rows in group Memory-B-Cells: 509879
Number of rows in group Plasma-B-Cells: 41614
Number of rows in group Naive-B-Cells: 666907
Number of rows in group RV+B-Cells: 771
Number of rows in group double-nagative-B-cells: 4826
Number of rows in group Plasmablast: 9248
Number of rows in group ASC: 1534
Number of rows in group CD27-memory-and-Plasmablast_Plasma-B-Cells: 31232
Number of rows in group Plasmablast_Plasma-B-Cells: 17257
Grouped data saved to CSV files.


### Memory B Cells

In [5]:
file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/OAS_data_grouped_by_BTypes/extra_cols/BType_Memory-B-Cells_extra_cols.csv" 
mem_subj_df = CSV.read(file_path, DataFrame);

print(mem_subj_df.Subject[1:100])

String15["Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "Donor-1", "

In [6]:
# Filter and print entries with "D" in their name for each column
println("Entries with 'D' in general_v_gene_heavy:")
println(filter(row -> occursin("D", row[:general_v_gene_heavy]), mem_subj_df)[:, :general_v_gene_heavy])

println("Entries with 'D' in general_v_gene_light:")
filter(row -> occursin("D", row[:general_v_gene_light]), mem_subj_df)[:, :general_v_gene_light]


Entries with 'D' in general_v_gene_heavy:
String15["IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-43D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-43D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", 

9007-element PooledArrays.PooledVector{String15, UInt32, Vector{UInt32}}:
 "IGKV1D-16"
 "IGKV1D-8"
 "IGKV2D-29"
 "IGKV2D-29"
 "IGKV3D-15"
 "IGKV3D-11"
 "IGKV1D-12"
 "IGKV3D-20"
 "IGKV6D-21"
 "IGKV1D-13"
 ⋮
 "IGKV1D-12"
 "IGKV1D-13"
 "IGKV3D-20"
 "IGKV3D-15"
 "IGKV1D-16"
 "IGKV1D-16"
 "IGKV3D-15"
 "IGKV3D-20"
 "IGKV3D-15"

In [7]:

# Add the new columns
mem_subj_df[!, :general_v_gene_heavy_no_para] = replace.(mem_subj_df.general_v_gene_heavy, r"D" => "")
mem_subj_df[!, :general_v_gene_light_no_para] = replace.(mem_subj_df.general_v_gene_light, r"D" => "")

# Display the modified DataFrame
println(first(mem_subj_df, 10))  # Print the first 10 rows for verification

10×25 DataFrame
 Row │ sequence_heavy                     locus_heavy  v_call_heavy   sequence_alignment_heavy           sequence_alignment_aa_heavy        germline_alignment_aa_heavy        cdr3_aa_heavy          sequence_light                     locus_light  v_call_light  sequence_alignment_light           sequence_alignment_aa_light        germline_alignment_aa_light        cdr3_aa_light  sequence_alignment_heavy_sep_light  BType           Disease  Species  Subject   Author             Age      general_v_gene_heavy  general_v_gene_light  general_v_gene_heavy_no_para  general_v_gene_light_no_para 
     │ String                             String1      String15       String                             String                             String                             String                 String                             String1      String15      String                             String                             String                             String31       String      

In [8]:
# Filter and print entries with "D" in their name for each column
println("Entries with 'D' in general_v_gene_heavy_no_para:")
println(filter(row -> occursin("D", row[:general_v_gene_heavy_no_para]), mem_subj_df)[:, :general_v_gene_heavy_no_para])

println("Entries with 'D' in general_v_gene_light_no_para:")
filter(row -> occursin("D", row[:general_v_gene_light_no_para]), mem_subj_df)[:, :general_v_gene_light_no_para]


Entries with 'D' in general_v_gene_heavy_no_para:
String[]
Entries with 'D' in general_v_gene_light_no_para:


String[]

In [9]:
mem_subjects_counts = countmap(mem_subj_df.Subject)

Dict{String15, Int64} with 15 entries:
  "Subject-BCP4" => 2090
  "S-CoV11"      => 21204
  "S-CoV10"      => 10915
  "S-CoV13"      => 20187
  "Subject-BCP5" => 5793
  "None"         => 17050
  "Donor-2"      => 140581
  "Subject-BCP9" => 3390
  "Subject-BCP8" => 1075
  "Donor-3"      => 44097
  "S-CoV1"       => 9210
  "Donor-4"      => 50177
  "Donor-1"      => 178545
  "Subject-BCP6" => 3358
  "Subject-BCP3" => 2207

In [10]:
# remove duplicates in mem_subj_df
mem_subj_df = unique(mem_subj_df)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String15,String,String7,String15,String31,String7,String15,String15,String,String
1,GGCTTTCTGAGAGTCATGGGCCTCCTGTGCAAGAACATGAAGCACCTGTGGTTTTTCCTCCTGCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTTAAGCCTTTGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTTATTACATCAGCGATGGTTACTTCTGGGCCTGGATCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGCTTGGGGCTCTCTATCATACTGGGACCACCTACTACAACCCGTCCCTCAAGCGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAACTTTCCCTGAGGCTGAACTCTGTGACCGCCGCAGACACGGCCGTGTATTATTGTGCGAGAGACCCAACCCCGGGAACGCCGGTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV4-38-2*02,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTTAAGCCTTTGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTTATTACATCAGCGATGGTTACTTCTGGGCCTGGATCCGGCAGCCCCCAGGGAAGGGGCTGGAGTGGCTTGGGGCTCTCTATCATACTGGGACCACCTACTACAACCCGTCCCTCAAGCGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAACTTTCCCTGAGGCTGAACTCTGTGACCGCCGCAGACACGGCCGTGTATTATTGTGCGAGAGACCCAACCCCGGGAACGCCGGTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,QVQLQESGPGLVKPLETLSLTCTVSGYYISDGYFWAWIRQPPGKGLEWLGALYHTGTTYYNPSLKRRVTISVDTSKNQLSLRLNSVTAADTAVYYCARDPTPGTPVDYWGQGTLVTVSS,QVQLQESGPGLVKPSETLSLTCTVSGYSISSGYYWGWIRQPPGKGLEWIGSIYHSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXGTXXDYWGQGTLVTVSS,ARDPTPGTPVDY,GGGAGGAATCAGTCCCACTCAGGACACAGCATGGACATGAGGGTCCCCGCTCAGCTCCTGGGGCTCCTGCTGCTCTGGTTCCCAGGTGCCAGGTGTGACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCAAGTCAGGGGATTAGAGATGATTTAGGCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAGCGCCTGATCTATGCTGCATCCAATTTGCAAAGTGGGGTCCCATCGAGGTTCAGCGGCAGTGGATCTGGGACAGAATTCACTCTCACAATCAGCAGCCTGCAGCCTGAAGATTCTGCAACTTATTACTGTCTACAGCATAATATTTACCCTCGGACGTTCGGCCAAGGGACCAGGGTGGAAATCAAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,K,IGKV1-17*01,GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCAAGTCAGGGGATTAGAGATGATTTAGGCTGGTATCAGCAGAAACCAGGGAAAGCCCCTAAGCGCCTGATCTATGCTGCATCCAATTTGCAAAGTGGGGTCCCATCGAGGTTCAGCGGCAGTGGATCTGGGACAGAATTCACTCTCACAATCAGCAGCCTGCAGCCTGAAGATTCTGCAACTTATTACTGTCTACAGCATAATATTTACCCTCGGACGTTCGGCCAAGGGACCAGGGTGGAAATCAAAC,DIQMTQSPSSLSASVGDRVTITCRASQGIRDDLGWYQQKPGKAPKRLIYAASNLQSGVPSRFSGSGSGTEFTLTISSLQPEDSATYYCLQHNIYPRTFGQGTRVEIK,DIQMTQSPSSLSASVGDRVTITCRASQGIRNDLGWYQQKPGKAPKRLIYAASSLQSGVPSRFSGSGSGTEFTLTISSLQPEDFATYYCLQHNSYPRTFGQGTKVEIK,LQHNIYPRT,QVQLQESGPGLVKPLETLSLTCTVSGYYISDGYFWAWIRQPPGKGLEWLGALYHTGTTYYNPSLKRRVTISVDTSKNQLSLRLNSVTAADTAVYYCARDPTPGTPVDYWGQGTLVTVSS[SEP]DIQMTQSPSSLSASVGDRVTITCRASQGIRDDLGWYQQKPGKAPKRLIYAASNLQSGVPSRFSGSGSGTEFTLTISSLQPEDSATYYCLQHNIYPRTFGQGTRVEIK,Memory-B-Cells,None,human,Donor-1,"Phad et al., 2022",no,IGHV4-38-2,IGKV1-17,IGHV4-38-2,IGKV1-17
2,ATAAGGGAAATGCTTTCTGAGAGTCATGGACCTCCTGTGCAAGAACATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGGTCCTGTCCCAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGCAGTAGTAGTTACTACTGGGGCTGGATCCGCCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATTATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCGGACACGGCCGTGTATTACTGTGCGAGAATACAAAGGATAGCAGCAGCTGGTACAACCCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCACCCTCCTCCAAGAGCACCTCTGGGGGCACAGCGGCCCTGGGCTGCCTGGTCAAGGACTACTTCCCCGAACCGGTGACGGTGTCGTGGAACTCAGGCGCCCTGACCAGCGGCGTGCACACCTTCCCGGCTGTCCTACAGTCCTCAGGA,H,IGHV4-39*07,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCT

In [11]:
# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
mem_subj_grouped_df = groupby(mem_subj_df, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
mem_subj_filtered_grouped = filter(g -> nrow(g) > 1, mem_subj_grouped_df)

# Step 2: Calculate the percentage of identical genes for each group
percentages = []

Any[]

In [12]:
group_sizes_mem_subj = [nrow(group) for group in mem_subj_filtered_grouped]

size_counts_mem_subj = combine(groupby(DataFrame(size = group_sizes_mem_subj), :size), nrow => :count)

println(size_counts_mem_subj)

118×2 DataFrame
 Row │ size   count 
     │ Int64  Int64 
─────┼──────────────
   1 │     2  28261
   2 │     3   7850
   3 │     4   3423
   4 │     5   1789
   5 │     6   1117
   6 │     7    713
   7 │     8    480
   8 │     9    363
   9 │    10    260
  10 │    11    174
  11 │    12    132
  12 │    13    109
  13 │    14     89
  14 │    15     72
  15 │    16     61
  16 │    17     51
  17 │    18     35
  18 │    19     37
  19 │    20     30
  20 │    21     20
  21 │    22     29
  22 │    23     14
  23 │    24     16
  24 │    25     18
  25 │    26     11
  26 │    27      9
  27 │    28     19
  28 │    29     14
  29 │    30     12
  30 │    31      8
  31 │    32      6
  32 │    33      4
  33 │    34      9
  34 │    35      7
  35 │    36      4
  36 │    37      4
  37 │    38      5
  38 │    39      5
  39 │    40      1
  40 │    41      2
  41 │    42      2
  42 │    43      5
  43 │    44      4
  44 │    45      6
  45 │    46      1
  46 │    47      3
 

In [13]:
# filter out every group that has the same entry in Subject
filtered_groups_only_sev_subj = filter(g -> length(unique(g.Subject)) > 1, mem_subj_filtered_grouped)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String15,String,String7,String15,String31,String7,String15,String15,String,String
1,GAGCTCTGGGAGAGGAGCCCAGCACTAGAAGTCGGCGGTGTTTCCATTCGGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGGTTTTCCTCGTTGCTCTTTTAAGAGGTGTCCAGTGTCAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTAGTAGCTTTGGCATGCACTGGGTCCGGCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGGCGGTCATATCATTTTCATTTATGGGAAGTTTTGAATACTATTCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAGCACGCTGTATCTGCAAATGGACAACCTGAGAGTTGAGGACACGGCTGTATATTACTGTGTGAAAGAATCGAATGCTTTTGATGTCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-30*18,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTAGTAGCTTTGGCATGCACTGGGTCCGGCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGGCGGTCATATCATTTTCATTTATGGGAAGTTTTGAATACTATTCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAGCACGCTGTATCTGCAAATGGACAACCTGAGAGTTGAGGACACGGCTGTATATTACTGTGTGAAAGAATCGAATGCTTTTGATGTCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAG,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSFGMHWVRQAPGKGLEWVAVISFSFMGSFEYYSDSVKGRFTISRDNSKSTLYLQMDNLRVEDTAVYYCVKESNAFDVWGQGTMVTVSS,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVISYDGSNKYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKXXXAFDVWGQGTMVTVSS,VKESNAFDV,GAAAACAGAGCTTCAGCAAGCATAGTGGGAATCTGCACCATGCCCTGGGCTCTGCTCCTCCTGACCCTCCTCACTCACTCTGCAGTGTCAGTGGTCCAGGCAGGGCTGACTCAGCCACCCTCGGTGTCCAAGGGCCTGAGACAGACCGCCACAGTCACCTGCACTGGGAACAGCAACAATGTTGGCGACCAAGGAGCCGCTTGGCTGCAGCAGCACCAGGGCCACCCTCCCAAACTCCTTTCCTACAGGAATAACAACCGGCCCTCAGGGATCTCAGAGAGATTATCTGCATCCAGGTCAGGAAACACAGCCTCCCTGACCATTACTGGACTCCAGCCTGAGGACGAGGCTGACTATTACTGCTCAGCATGGGACAGTAGACTCAATGTTTGGGTGTTCGGCGGAGGGACCAAGCTGACCGTCCTAGGTCAGCCCAAGGCTGCCCCCTCGGTCACTCTGTTCCCACCCTCCTCTGAGGAGCTTCAAGCCAACAAGGCCACACTGGTGTGTCTCATAAGTGACTTCTACCCGGGAGCCGTGACAGTGGCCTGGAAGGCAGATAGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCACACCCTCCAAACAAAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV10-54*01,CAGGCAGGGCTGACTCAGCCACCCTCGGTGTCCAAGGGCCTGAGACAGACCGCCACAGTCACCTGCACTGGGAACAGCAACAATGTTGGCGACCAAGGAGCCGCTTGGCTGCAGCAGCACCAGGGCCACCCTCCCAAACTCCTTTCCTACAGGAATAACAACCGGCCCTCAGGGATCTCAGAGAGATTATCTGCATCCAGGTCAGGAAACACAGCCTCCCTGACCATTACTGGACTCCAGCCTGAGGACGAGGCTGACTATTACTGCTCAGCATGGGACAGTAGACTCAATGTTTGGGTGTTCGGCGGAGGGACCAAGCTGACCGTCCTAG,QAGLTQPPSVSKGLRQTATVTCTGNSNNVGDQGAAWLQQHQGHPPKLLSYRNNNRPSGISERLSASRSGNTASLTITGLQPEDEADYYCSAWDSRLNVWVFGGGTKLTVL,QAGLTQPPSVSKGLRQTATLTCTGNSNNVGNQGAAWLQQHQGHPPKLLSYRNNNRPSGISERLSASRSGNTASLTITGLQPEDEADYYCSAWDSSLSXWVFGGGTKLTVL,SAWDSRLNVWV,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSFGMHWVRQAPGKGLEWVAVISFSFMGSFEYYSDSVKGRFTISRDNSKSTLYLQMDNLRVEDTAVYYCVKESNAFDVWGQGTMVTVSS[SEP]QAGLTQPPSVSKGLRQTATVTCTGNSNNVGDQGAAWLQQHQGHPPKLLSYRNNNRPSGISERLSASRSGNTASLTITGLQPEDEADYYCSAWDSRLNVWVFGGGTKLTVL,Memory-B-Cells,None,human,Donor-1,"Phad et al., 2022",no,IGHV3-30,IGLV10-54,IGHV3-30,IGLV10-54
2,GGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGGTTTTCCTCGTTGCTCTTTTAAGAGGTGTCCAATGTCAGGTGCAGCTGGCGGAGTCTGGGGGAGGCTTAGTCCAGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTAACTCTCAGTAATTATGTCACGCACTGGGTCCGCCAGGCTCCAGGCAAGGGACCGGAGTGGCTGGCAGATATGTCACAGGATGGAAGTATTAGAATGTATGCAGGCTCCGTGAGGGGCCGATTCACCATCTCTAGAGACAATTCCGAGAACACTCTCTATCTGCAAATGAACAGCCTGAGAGTTGAGGACACGGGTGTATATTACTGTGTGAAAGAATCCAATGCTTTTGATGTCTGGGGCCAAGGGACAATGGTCATCGTCTCTTCAGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCGCCCTGCTCCAGGAGCACCTCCGAGAGCACAGCGGCCCTGGGCTGCCTGGTCAAGGACTACTTCCCCGAACCGGTGACGGTGTCGTGGAACTCAGGCGCTCTGACCAGCGGCGTGCACACCTTCCCAGCTGTCCTACAGTCCTCAGGA,H

In [14]:
# Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases = 0
total_groups = length(filtered_groups_only_sev_subj)

for group in filtered_groups_only_sev_subj
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes = unique(group.general_v_gene_light_no_para)
    if length(unique_genes) == 1
        true_cases += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true = (true_cases / total_groups) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true%")
fraction_true

Percentage of groups where all entries have the same general_v_gene_light: 55.8169375534645%


55.8169375534645

## only with data from Jaffe et al. 2022

In [15]:
mem_subj_df_jaffe_only = filter(g -> g.Author == "Jaffe et al., 2022", mem_subj_df)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String15,String,String7,String15,String31,String7,String15,String15,String,String
1,CCACATCCCTCCTCAGAAGCCCCCAGAGCACAACGCCTCACCATGGACTGGACCTGGAGGATCCTCTTTTTGGTGGCAGCAGCCACAGGTGCCCACTCCCAGGTCCAACTTGTGCAGTCTGGGGCTGAGGTGAGGAAGCCTGGGGCCTCAGTGAAGGTTTCCTGCAAGGCTTCTGGATACACCTTCACTACTTATGCTATACATTGGGTGCGCCGGGCCCCCGGACAAAGGCTTGAGTGGATGGGATGGATCAACGCTGCCAATGGTAACACAGAATATTCACAGAGGTTCCAGGGCAGAGTCACATTTACAAGGGACACATCCGCGACCACAGCCTACATGGAACTGAGGAGCCTGAGATCTGAAGACACGGCTATTTATTATTGTGCGGGAGACTCTCTGGCAGCAGATGGCCAATTCTTTGATTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV1-3*01,CAGGTCCAACTTGTGCAGTCTGGGGCTGAGGTGAGGAAGCCTGGGGCCTCAGTGAAGGTTTCCTGCAAGGCTTCTGGATACACCTTCACTACTTATGCTATACATTGGGTGCGCCGGGCCCCCGGACAAAGGCTTGAGTGGATGGGATGGATCAACGCTGCCAATGGTAACACAGAATATTCACAGAGGTTCCAGGGCAGAGTCACATTTACAAGGGACACATCCGCGACCACAGCCTACATGGAACTGAGGAGCCTGAGATCTGAAGACACGGCTATTTATTATTGTGCGGGAGACTCTCTGGCAGCAGATGGCCAATTCTTTGATTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,QVQLVQSGAEVRKPGASVKVSCKASGYTFTTYAIHWVRRAPGQRLEWMGWINAANGNTEYSQRFQGRVTFTRDTSATTAYMELRSLRSEDTAIYYCAGDSLAADGQFFDYWGQGTLVTVSS,QVQLVQSGAEVKKPGASVKVSCKASGYTFTSYAMHWVRQAPGQRLEWMGWINAGNGNTKYSQKFQGRVTITRDTSASTAYMELSSLRSEDTAVYYCARXXXAAAGXXFDYWGQGTLVTVSS,AGDSLAADGQFFDY,GGAGAAGAGCTGCTCAGTTAGGACCCAGAGGGAACCATGGAAACCCCAGCGCAGCTTCTCTTCCTCCTGCTACTCTGGCTCCCAGATGCCACCGGAGAAGTTGTGTTGACGCAGTCTCCAGGCACCCTGTCTTTGTCTCCAGGGGAAAGAGCCACCCTCTCCTGCAGGGCCAGTCAGGGTGTTAAGTACAGTTACTTAGCCTGGTACCAGCAGAAACCTGGCCAGGCTCCCAGGCTCCTCATCTATGGTGTTTCCAGCAGGGCCACTGGCATCCCAGACAGGTTCAGTGGCAGTGGGTCTGGGACAGACTTCACTCTCACCATCAGCAGACTGGAGCCTGAAGATTTTGCAGTGTATTACTGTCAGCAGTATGGTACCTCATCCACTTTCGGCCCTGGGACCAAGGTGGATATCAAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,K,IGKV3-20*01,GAAGTTGTGTTGACGCAGTCTCCAGGCACCCTGTCTTTGTCTCCAGGGGAAAGAGCCACCCTCTCCTGCAGGGCCAGTCAGGGTGTTAAGTACAGTTACTTAGCCTGGTACCAGCAGAAACCTGGCCAGGCTCCCAGGCTCCTCATCTATGGTGTTTCCAGCAGGGCCACTGGCATCCCAGACAGGTTCAGTGGCAGTGGGTCTGGGACAGACTTCACTCTCACCATCAGCAGACTGGAGCCTGAAGATTTTGCAGTGTATTACTGTCAGCAGTATGGTACCTCATCCACTTTCGGCCCTGGGACCAAGGTGGATATCAAAC,EVVLTQSPGTLSLSPGERATLSCRASQGVKYSYLAWYQQKPGQAPRLLIYGVSSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQYGTSSTFGPGTKVDIK,EIVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQYGSSXTFGPGTKVDIK,QQYGTSST,QVQLVQSGAEVRKPGASVKVSCKASGYTFTTYAIHWVRRAPGQRLEWMGWINAANGNTEYSQRFQGRVTFTRDTSATTAYMELRSLRSEDTAIYYCAGDSLAADGQFFDYWGQGTLVTVSS[SEP]EVVLTQSPGTLSLSPGERATLSCRASQGVKYSYLAWYQQKPGQAPRLLIYGVSSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQYGTSSTFGPGTKVDIK,Memory-B-Cells,SARS-COV-2,human,Donor-1,"Jaffe et al., 2022",45,IGHV1-3,IGKV3-20,IGHV1-3,IGKV3-20
2,AGCTCTCAGAGAGGTGCCTTAGCCCTGGATTCCAAGGCATTTCCACTTGGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTGGGGCTGTGCTGGGTTTTCCTTGTTGCTATTTTAGAAGGTGTCCAGTGTGAGGTGCAACTGGTGGAGTCTGGGGGAGGCTTGGTACAGCCGGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCGCCTTCAGTAGATATAGCATGAACTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTTCATACATTCGTAGTAGTGGTAGTCCCGTATACTACGCAGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAATGCCAAGAACTTACTGTTTCTACAAATGAACAGCCTGAGAGCCGAGGACACGGCTGTTTATTACTGTGCGAGAGATCTACCCATGAATGCTTTTGATATTTGGGGCCCAGGGACAATGGTCACCGTCTCTTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-48*01,GAGGTGCAACTGGTGGAGTCTGGGGGAGGCTTGGTACAGCCGGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCGCCTTCAGTAGATATAGCATGAACTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTTCATACAT

In [16]:
# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
mem_subj_grouped_df_jaffe_only = groupby(mem_subj_df_jaffe_only, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
mem_subj_filtered_grouped_jaffe_only = filter(g -> nrow(g) > 1, mem_subj_grouped_df_jaffe_only)

# filter out every group that has the same entry in Subject
filtered_groups_only_sev_subj_jaffe_only = filter(g -> length(unique(g.Subject)) > 1, mem_subj_filtered_grouped_jaffe_only)


Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String15,String,String7,String15,String31,String7,String15,String15,String,String
1,AGGTCTCAGAGAGGAGCCTTAGCCCTGGACTCCAAGGCCTTTCCACTTGGTGATCAGCACTGAGCACAGAGGACTCACCATGGAATTGGGGCTGAGCTGGGTTTTCCTTGTTGCTATTTTAGAAGGTGTCCAGTGTCAGGTGCAACTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCGGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCATCTTTAGTAGTTATAAGATGAGTTGGGTCCGGCAAATTCCAGGGAAGGGGCTGGAATGGGTGGCCAACATAATGCAAGATGGGAGTGGGCGGGACTATGTGGACTCTGTGCAGGGCCGATTCACTATCTCCAGAGATAACGCCAAGAATTCACTGTATCTGCAAATGAACAGCCTGAGAGTCGAGGACACGGCTGTGTATTACTGTGCGGGTTGGGCTGGCAGCAGTTGGTTTGACTACTGGGGCCAGGGAACCCTGGTCGTCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-7*01,AGGTGCAACTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCGGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCATCTTTAGTAGTTATAAGATGAGTTGGGTCCGGCAAATTCCAGGGAAGGGGCTGGAATGGGTGGCCAACATAATGCAAGATGGGAGTGGGCGGGACTATGTGGACTCTGTGCAGGGCCGATTCACTATCTCCAGAGATAACGCCAAGAATTCACTGTATCTGCAAATGAACAGCCTGAGAGTCGAGGACACGGCTGTGTATTACTGTGCGGGTTGGGCTGGCAGCAGTTGGTTTGACTACTGGGGCCAGGGAACCCTGGTCGTCGTCTCCTCAG,VQLVESGGGLVQPGGSLRLSCAASGFIFSSYKMSWVRQIPGKGLEWVANIMQDGSGRDYVDSVQGRFTISRDNAKNSLYLQMNSLRVEDTAVYYCAGWAGSSWFDYWGQGTLVVVSS,VQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMSWVRQAPGKGLEWVANIKQDGSEKYYVDSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCAXXXXSSWFDYWGQGTLVTVSS,AGWAGSSWFDY,TCTGGCACCAGGGGTCCCTTCCAATATCAGCACCATGGCCTGGACTCCTCTCTTTCTGTTCCTCCTCACTTGCTGCCCAGGGTCCAATTCCCAGGCTGTGGTGACTCAGGAGCCCTCACTGACTGTGTCCCCAGGAGGGACAGTCACTCTCACCTGTGGCTCCAGCACTGGACCTGTCACCAGTACTCACTATCCCTACTGGTTCCAGCAGAAGCCTGGCCAAGCCCCCAGGACACTGATTTATGATACAGCCAACAAACACTCCTGGACACCTGCCCGATTCTCAGGCTCCCTCCTTGGGGACAAAGCTGCCCTGACCCTTTCGGGTGCGCAGCCTGAGGATGAAGCTGAATATTACTGCTTGCTCTCCTATAGCGGTCCTCGGGGTCCTCGGGTGGTATTCGGCGGAGGGACCAGGCTGACCGTCCTAGGTCAGCCCAAGGCTGCCCCCTCGGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTTCAAGCCAACAAGGCCACACTGGTGTGTCTCATAAGTGACTTCTACCCGGGAGCCGTGACAGTGGCCTGGAAGGCAGATAGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCACACCCTCCAAACAAAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV7-46*01,CAGGCTGTGGTGACTCAGGAGCCCTCACTGACTGTGTCCCCAGGAGGGACAGTCACTCTCACCTGTGGCTCCAGCACTGGACCTGTCACCAGTACTCACTATCCCTACTGGTTCCAGCAGAAGCCTGGCCAAGCCCCCAGGACACTGATTTATGATACAGCCAACAAACACTCCTGGACACCTGCCCGATTCTCAGGCTCCCTCCTTGGGGACAAAGCTGCCCTGACCCTTTCGGGTGCGCAGCCTGAGGATGAAGCTGAATATTACTGCTTGCTCTCCTATAGCGGTCCTCGGGGTCCTCGGGTGGTATTCGGCGGAGGGACCAGGCTGACCGTCCTAG,QAVVTQEPSLTVSPGGTVTLTCGSSTGPVTSTHYPYWFQQKPGQAPRTLIYDTANKHSWTPARFSGSLLGDKAALTLSGAQPEDEAEYYCLLSYSGPRGPRVVFGGGTRLTVL,QAVVTQEPSLTVSPGGTVTLTCGSSTGAVTSGHYPYWFQQKPGQAPRTLIYDTSNKHSWTPARFSGSLLGGKAALTLSGAQPEDEAEYYCLLSYSGARXXXVVFGGGTKLTVL,LLSYSGPRGPRVV,VQLVESGGGLVQPGGSLRLSCAASGFIFSSYKMSWVRQIPGKGLEWVANIMQDGSGRDYVDSVQGRFTISRDNAKNSLYLQMNSLRVEDTAVYYCAGWAGSSWFDYWGQGTLVVVSS[SEP]QAVVTQEPSLTVSPGGTVTLTCGSSTGPVTSTHYPYWFQQKPGQAPRTLIYDTANKHSWTPARFSGSLLGDKAALTLSGAQPEDEAEYYCLLSYSGPRGPRVVFGGGTRLTVL,Memory-B-Cells,SARS-COV-2,human,Donor-1,"Jaffe et al., 2022",45,IGHV3-7,IGLV7-46,IGHV3-7,IGLV7-46
2,AGGTCTCAGAGAGGAGCCTTAGCCCTGGACTCCAAGGCCTTTCCACTTGGTGATCAGCACTGAGCACAGAGGACTCACCATGGAATTGGGGCTGAGCTGGGTTTTCCTTGTTGCTATTTTAGAAGGTGTCCAGTGTCAGGTGCAACTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCGGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCATCTTTAGTAGTTATAAGATGAGTTGGGTCCGGCAAATTCCAGGGAAGGGGCTGGAATGGGTGGCCAACATAATGCAAGATGGGAGTGGGCGGGACTATGTGGACTCTGTGCAGGGCCGATTCACTATCTCCAGAGATAACGCCAAGAATTCACTGTATCTGCAAATGAACAGCCTGAGAGTCGAGGACACGGCTGTGTATTACTGTGCGGGTTGGGCTGGCAGCAGTTGGTTTGACTACTGGGGCCAGGGAACCCTGGTCGTCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-7*01,AGGTGCAACTGGTGGAGTCTG

In [33]:
# alternative: Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases_jaffe_only = 0
total_groups_jaffe_only = length(filtered_groups_only_sev_subj_jaffe_only)

for group in filtered_groups_only_sev_subj_jaffe_only
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes_jaffe_only = unique(group.general_v_gene_light_no_para)
    if length(unique_genes_jaffe_only) == 1
        true_cases_jaffe_only += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true_jaffe_only = (true_cases_jaffe_only / total_groups_jaffe_only) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true_jaffe_only%")
fraction_true_jaffe_only

Percentage of groups where all entries have the same general_v_gene_light: 81.90045248868778%


81.90045248868778

# Data with Jaffe et al., 2022 excluded

In [17]:
mem_subj_df_no_jaffe = filter(g -> g.Author != "Jaffe et al., 2022", mem_subj_df)

# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
mem_subj_grouped_df_no_jaffe = groupby(mem_subj_df_no_jaffe, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
mem_subj_filtered_grouped_no_jaffe = filter(g -> nrow(g) > 1, mem_subj_grouped_df_no_jaffe)

# filter out every group that has the same entry in Subject
filtered_groups_only_sev_subj_no_jaffe = filter(g -> length(unique(g.Subject)) > 1, mem_subj_filtered_grouped_no_jaffe)


Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String15,String,String7,String15,String31,String7,String15,String15,String,String
1,AGCTCTGGGAGAGGAGCCCCAGCCTTGGGATTCCCAAGTGTTTTTATTCAGTGATCAGGACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGATTTTCCTTGCTGCTATTTTAAAAGGTGTCCAGTGTGAGGTGCACATGGTGGAATCTGGGGGAGGCTTGGTAAAGCCTGGGGGGTCCCTTAGCCTCTCCTGTGCAGCCTCTGAATTCACTTTCAGTAAGGCCTGGATGACCTGGGTCCGCCAGGCTCCAGGGAAGGGGTTGGAATGGGTTGGCCGTATTAAAAGAAAAATTGATGGTGAGACAACAGACTACGCTGCACCCGTGAGAGGCAGATTCACCATCTCAAGAGATGATTCAAAAAACACTCTGTATCTACACATGAACAGCCTGAGAACCGAGGACACAGCCGTATATTACTGTGCCACAGGTTTGATTAATGCTTTTGATATCTGGGGCCAAGGGACACTGGTCACCGTCTCTTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCTGCAGCACCCAGCCAGATGGGAACGTGGTCATCGCCTGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGGGCGTGACCGCCAGAAACTTCCC,H,IGHV3-15*01,GAGGTGCACATGGTGGAATCTGGGGGAGGCTTGGTAAAGCCTGGGGGGTCCCTTAGCCTCTCCTGTGCAGCCTCTGAATTCACTTTCAGTAAGGCCTGGATGACCTGGGTCCGCCAGGCTCCAGGGAAGGGGTTGGAATGGGTTGGCCGTATTAAAAGAAAAATTGATGGTGAGACAACAGACTACGCTGCACCCGTGAGAGGCAGATTCACCATCTCAAGAGATGATTCAAAAAACACTCTGTATCTACACATGAACAGCCTGAGAACCGAGGACACAGCCGTATATTACTGTGCCACAGGTTTGATTAATGCTTTTGATATCTGGGGCCAAGGGACACTGGTCACCGTCTCTTCAG,EVHMVESGGGLVKPGGSLSLSCAASEFTFSKAWMTWVRQAPGKGLEWVGRIKRKIDGETTDYAAPVRGRFTISRDDSKNTLYLHMNSLRTEDTAVYYCATGLINAFDIWGQGTLVTVSS,EVQLVESGGGLVKPGGSLRLSCAASGFTFSNAWMSWVRQAPGKGLEWVGRIKSKTDGGTTDYAAPVKGRFTISRDDSKNTLYLQMNSLKTEDTAVYYCTTXXINAFDIWGQGTMVTVSS,ATGLINAFDI,GGCTGGGGTCTCAGGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTATTCCTCACCCTCCTCACTCAGGGCACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCTGCCTCCGTGTGTGGGTCTCTTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGACGTTGGTGCTTATAACTATGTCTCCTGGTACCAACAACACCCAGGCAAAGCCCCCAAAGTCATGATTTATGATGTCAGTAATCGGCCCTCAGGGGTTTCTAATCGCTTCTCTGGCTCCAAGTCTGCCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGACGAGGCTGATTATTACTGCTGCTCATATACACTCACCAATACTGTGGTATTCGGCGGCGGGACCAAGCTGACCGTCCTAGGTCAGCCCAAGGCTGCCCCCTCGGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTTCAAGCCAACAAGGCCACACTGGTGTGTCTCATAAGTGACTTCTACCCGGGAGCCGTGACAGTGGCCTGGAAGGCAGATAGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCACACCCTCCAAACAAAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV2-14*03,CAGTCTGCCCTGACTCAGCCTGCCTCCGTGTGTGGGTCTCTTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGACGTTGGTGCTTATAACTATGTCTCCTGGTACCAACAACACCCAGGCAAAGCCCCCAAAGTCATGATTTATGATGTCAGTAATCGGCCCTCAGGGGTTTCTAATCGCTTCTCTGGCTCCAAGTCTGCCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGACGAGGCTGATTATTACTGCTGCTCATATACACTCACCAATACTGTGGTATTCGGCGGCGGGACCAAGCTGACCGTCCTAG,QSALTQPASVCGSLGQSITISCTGTSSDVGAYNYVSWYQQHPGKAPKVMIYDVSNRPSGVSNRFSGSKSANTASLTISGLQAEDEADYYCCSYTLTNTVVFGGGTKLTVL,QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSSTVVFGGGTKLTVL,CSYTLTNTVV,EVHMVESGGGLVKPGGSLSLSCAASEFTFSKAWMTWVRQAPGKGLEWVGRIKRKIDGETTDYAAPVRGRFTISRDDSKNTLYLHMNSLRTEDTAVYYCATGLINAFDIWGQGTLVTVSS[SEP]QSALTQPASVCGSLGQSITISCTGTSSDVGAYNYVSWYQQHPGKAPKVMIYDVSNRPSGVSNRFSGSKSANTASLTISGLQAEDEADYYCCSYTLTNTVVFGGGTKLTVL,Memory-B-Cells,None,human,Donor-1,"Phad et al., 2022",no,IGHV3-15,IGLV2-14,IGHV3-15,IGLV2-14
2,AGCTCTGGGAGAGGAGCCCCAGCCTTGGGATTCCCAGGTCTTTTCATTCAGTGATCAGGACTGAACACAGAGGACTCACCATGGACTTTGGGCTGAACTGGATTTTCCTTGCTGCTATTTTAAAAGGTGTCCAGTGTGAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTAAAGCCTGGGGGGTCCCTTAGACTCTCCTGTGCAGCCTCTGGATTCACTTTCAGTAGTGTCTGGATGACCTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTGGCCGTGTTAAAAGGAAAACTGATGGTGAGACAACAGACTACGCTGCACCCGTGAAAGGCAGATTCACCATCTCAAGAGATGATTTAAAAAACACGCTGTATCTGCAAATGAACAGCCTGAAAACCGAGGACACAGCCGTGTATTACTGTGCCACAGGTCTCATCAATGCTTTTGATATCTGGGGCCAAGGGACAGTGGTCACCGTCTCTTCAGGGAGTGCATCCGCCCCAACCCTTTTC

In [35]:
# alternative: Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases_no_jaffe = 0
total_groups_no_jaffe = length(filtered_groups_only_sev_subj_no_jaffe)

for group in filtered_groups_only_sev_subj_no_jaffe
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes_no_jaffe = unique(group.general_v_gene_light_no_para)
    if length(unique_genes_no_jaffe) == 1
        true_cases_no_jaffe += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true_no_jaffe = (true_cases_no_jaffe / total_groups_no_jaffe) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true_no_jaffe%")
fraction_true_no_jaffe

Percentage of groups where all entries have the same general_v_gene_light: 51.358277806253206%


51.358277806253206

## Naive B Cells

In [18]:
file_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/coherence_analysis_in_oas_db/data/OAS_data_grouped_by_BTypes/extra_cols/BType_Naive-B-Cells_extra_cols.csv" 
naive_subj_df = CSV.read(file_path, DataFrame);

print(naive_subj_df.Subject[1:100])

String15["Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "Donor-4", "

In [19]:
# Filter and print entries with "D" in their name for each column
println("Entries with 'D' in general_v_gene_heavy:")
println(filter(row -> occursin("D", row[:general_v_gene_heavy]), naive_subj_df)[:, :general_v_gene_heavy])

println("Entries with 'D' in general_v_gene_light:")
println(filter(row -> occursin("D", row[:general_v_gene_light]), naive_subj_df)[:, :general_v_gene_light])


Entries with 'D' in general_v_gene_heavy:
String15["IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", "IGHV3-64D", 

In [20]:

# Add the new columns
naive_subj_df[!, :general_v_gene_heavy_no_para] = replace.(naive_subj_df.general_v_gene_heavy, r"D" => "")
naive_subj_df[!, :general_v_gene_light_no_para] = replace.(naive_subj_df.general_v_gene_light, r"D" => "")

# Display the modified DataFrame
println(first(naive_subj_df, 10))  # Print the first 10 rows for verification

10×25 DataFrame
 Row │ sequence_heavy                     locus_heavy  v_call_heavy  sequence_alignment_heavy           sequence_alignment_aa_heavy        germline_alignment_aa_heavy        cdr3_aa_heavy                 sequence_light                     locus_light  v_call_light  sequence_alignment_light           sequence_alignment_aa_light        germline_alignment_aa_light        cdr3_aa_light  sequence_alignment_heavy_sep_light  BType          Disease   Species  Subject   Author              Age    general_v_gene_heavy  general_v_gene_light  general_v_gene_heavy_no_para  general_v_gene_light_no_para 
     │ String                             String1      String15      String                             String                             String                             String                        String                             String1      String15      String                             String                             String                             String31       S

In [21]:
# Filter and print entries with "D" in their name for each column
println("Entries with 'D' in general_v_gene_heavy:")
println(filter(row -> occursin("D", row[:general_v_gene_heavy_no_para]), naive_subj_df)[:, :general_v_gene_heavy_no_para])

println("Entries with 'D' in general_v_gene_light:")
filter(row -> occursin("D", row[:general_v_gene_light_no_para]), naive_subj_df)[:, :general_v_gene_light_no_para]


Entries with 'D' in general_v_gene_heavy:
String[]
Entries with 'D' in general_v_gene_light:


String[]

In [22]:
naive_subj_df = unique(naive_subj_df)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String15,String15,String7,String15,String31,Int64,String15,String15,String,String
1,ACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGTAGTTACTACTGGAGCTGGATCCGGCAGCCCCCAGGGAAGGGACTGGAGTGGATTGGGTATATCTATTACAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCTGCGGACACGGCCGTGTATTACTGTGCGAGAGATGAACTTGGGGGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV4-59*01,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGTAGTTACTACTGGAGCTGGATCCGGCAGCCCCCAGGGAAGGGACTGGAGTGGATTGGGTATATCTATTACAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCTGCGGACACGGCCGTGTATTACTGTGCGAGAGATGAACTTGGGGGTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARDELGGDYWGQGTLVTVSS,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXGGDYWGQGTLVTVSS,ARDELGGDY,AGCTTCAGCTGTGGGTAGAGAAGACAGGACTCAGGACAATCTCCAGCATGGCCAGCTTCCCTCTCCTCCTCACCCTCCTCACTCACTGTGCAGGGTCCTGGGCCCAGTCTGTGCTGACTCAGCCACCCTCAGCGTCTGGGACCCCCGGGCAGAGGGTCACCATCTCTTGTTCTGGAAGCAGCTCCAACATCGGAAGTAATACTGTAAACTGGTACCAGCAGCTCCCAGGAACGGCCCCCAAACTCCTCATCTATAGTAATAATCAGCGGCCCTCAGGGGTCCCTGACCGATTCTCTGGCTCCAAGTCTGGCACCTCAGCCTCCCTGGCCATCAGTGGGCTCCAGTCTGAGGATGAGGCTGATTATTACTGTGCAGCATGGGATGACAGCCTGAATGGTCGAGTGTTCGGCGGAGGGACCAAGCTGACCGTCCTAGGTCAGCCCAAGGCTGCCCCCTCGGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTTCAAGCCAACAAGGCCACACTGGTGTGTCTCATAAGTGACTTCTACCCGGGAGCCGTGACAGTGGCCTGGAAGGCAGATAGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCACACCCTCCAAACAAAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV1-44*01,CAGTCTGTGCTGACTCAGCCACCCTCAGCGTCTGGGACCCCCGGGCAGAGGGTCACCATCTCTTGTTCTGGAAGCAGCTCCAACATCGGAAGTAATACTGTAAACTGGTACCAGCAGCTCCCAGGAACGGCCCCCAAACTCCTCATCTATAGTAATAATCAGCGGCCCTCAGGGGTCCCTGACCGATTCTCTGGCTCCAAGTCTGGCACCTCAGCCTCCCTGGCCATCAGTGGGCTCCAGTCTGAGGATGAGGCTGATTATTACTGTGCAGCATGGGATGACAGCCTGAATGGTCGAGTGTTCGGCGGAGGGACCAAGCTGACCGTCCTAG,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPGTAPKLLIYSNNQRPSGVPDRFSGSKSGTSASLAISGLQSEDEADYYCAAWDDSLNGRVFGGGTKLTVL,QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPGTAPKLLIYSNNQRPSGVPDRFSGSKSGTSASLAISGLQSEDEADYYCAAWDDSLNGXVFGGGTKLTVL,AAWDDSLNGRV,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARDELGGDYWGQGTLVTVSS[SEP]QSVLTQPPSASGTPGQRVTISCSGSSSNIGSNTVNWYQQLPGTAPKLLIYSNNQRPSGVPDRFSGSKSGTSASLAISGLQSEDEADYYCAAWDDSLNGRVFGGGTKLTVL,Naive-B-Cells,CMV,human,Donor-4,"Jaffe et al., 2022",50,IGHV4-59,IGLV1-44,IGHV4-59,IGLV1-44
2,ACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGTAGTTACTACTGGAGCTGGATCCGGCAGCCCCCAGGGAAGGGACTGGAGTGGATTGGGTATATCTATTACAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCATATCAGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCTGCGGACACGGCCGTGTATTACTGTGCGGGGTCCCGGCCCAGAGTGTATTACGATTTTTGGAGTGGTTACCCCTACTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV4-59*01,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGTAGTTACTACTGGAGCTGGATCCGGCAGCCCCCAGGGAAGGGACTGGAGT

In [23]:
naive_subjects_counts = countmap(naive_subj_df.Subject)

Dict{String15, Int64} with 5 entries:
  "Donor-4"   => 183643
  "Patient-1" => 1896
  "Donor-2"   => 149346
  "Donor-1"   => 166539
  "Donor-3"   => 161158

In [24]:
# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
naive_subj_grouped_df = groupby(naive_subj_df, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
naive_subj_filtered_grouped = filter(g -> nrow(g) > 1, naive_subj_grouped_df)


Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String15,String15,String7,String15,String31,Int64,String15,String15,String,String
1,GGGGGCTTTCTGAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGGTCCTGTCCCAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGCAGTAGTAGTTACTACTGGGGCTGGATCCGCCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATTATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCCGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCTGTGTATTACTGTGCGAGAATTACTATGATAGTAGGGTACAATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV4-39*01,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGCAGTAGTAGTTACTACTGGGGCTGGATCCGCCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATTATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCCGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCTGTGTATTACTGTGCGAGAATTACTATGATAGTAGGGTACAATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAG,QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARITMIVGYNAFDIWGQGTMVTVSS,QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARITMIVXXXAFDIWGQGTMVTVSS,ARITMIVGYNAFDI,GGGGGGGGTCTCAGGAGGCAGCACTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTCCTCCTCAGCCTCCTCACTCAGGGCACAGGATCCTGGGCTCAGTCTGCCCTGACTCAGCCTCGCTCAGTGTCCGGGTCTCCTGGACAGTCAGTCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGTGGTTATAACTATGTCTCCTGGTACCAACAGCACCCAGGCAAAGCCCCCAAACTCATGATTTATGATGTCAGTAAGCGGCCCTCAGGGGTCCCTGATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGATGAGGCTGATTATTACTGCTGCTCATATGCAGGCAGCTACACTGTGGTATTCGGCGGAGGGACCAAGCTGACCGTCCTAGGTCAGCCCAAGGCTGCCCCCTCGGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTTCAAGCCAACAAGGCCACACTGGTGTGTCTCATAAGTGACTTCTACCCGGGAGCCGTGACAGTGGCCTGGAAGGCAGATAGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCACACCCTCCAAACAAAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV2-11*01,CAGTCTGCCCTGACTCAGCCTCGCTCAGTGTCCGGGTCTCCTGGACAGTCAGTCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGTGGTTATAACTATGTCTCCTGGTACCAACAGCACCCAGGCAAAGCCCCCAAACTCATGATTTATGATGTCAGTAAGCGGCCCTCAGGGGTCCCTGATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGATGAGGCTGATTATTACTGCTGCTCATATGCAGGCAGCTACACTGTGGTATTCGGCGGAGGGACCAAGCTGACCGTCCTAG,QSALTQPRSVSGSPGQSVTISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSKRPSGVPDRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSYTVVFGGGTKLTVL,QSALTQPRSVSGSPGQSVTISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSKRPSGVPDRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSYTVVFGGGTKLTVL,CSYAGSYTVV,QLQLQESGPGLVKPSETLSLTCTVSGGSISSSSYYWGWIRQPPGKGLEWIGSIYYSGSTYYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARITMIVGYNAFDIWGQGTMVTVSS[SEP]QSALTQPRSVSGSPGQSVTISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSKRPSGVPDRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSYTVVFGGGTKLTVL,Naive-B-Cells,CMV,human,Donor-4,"Jaffe et al., 2022",50,IGHV4-39,IGLV2-11,IGHV4-39,IGLV2-11
2,TGGGGGCTTTCTGAGAGTCATGGATCTCATGTGCAAGAAAATGAAGCACCTGTGGTTCTTCCTCCTGCTGGTGGCGGCTCCCAGATGGGTCCTGTCCCAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGCAGTAGTAGTTACTACTGGGGCTGGATCCGCCAGCCCCCAGGGAAGGGGCTGGAGTGGATTGGGAGTATCTATTATAGTGGGAGCACCTACTACAACCCGTCCCTCAAGAGTCGAGTCACCATATCCGTAGACACGTCCAAGAACCAGTTCTCCCTGAAGCTGAGCTCTGTGACCGCCGCAGACACGGCTGTGTATTACTGTGCGAGAATTACTATGATAGTAGGGTACAATGCTTTTGATATCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV4-39*01,CAGCTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGT

In [25]:
group_sizes_naive_subj = [nrow(group) for group in naive_subj_filtered_grouped]

size_counts_naive_subj = combine(groupby(DataFrame(size = group_sizes_naive_subj), :size), nrow => :count)

println(size_counts_naive_subj)

33×2 DataFrame
 Row │ size   count 
     │ Int64  Int64 
─────┼──────────────
   1 │     2   6855
   2 │     3    978
   3 │     4    389
   4 │     5    241
   5 │     6    125
   6 │     7     82
   7 │     8     48
   8 │     9     32
   9 │    10     38
  10 │    11     21
  11 │    12      9
  12 │    13     15
  13 │    14      7
  14 │    15      4
  15 │    16      5
  16 │    17      5
  17 │    18      3
  18 │    19      2
  19 │    21      3
  20 │    22      1
  21 │    25      4
  22 │    26      2
  23 │    27      1
  24 │    28      1
  25 │    29      2
  26 │    30      1
  27 │    32      2
  28 │    34      2
  29 │    44      1
  30 │    45      1
  31 │    56      1
  32 │    58      1
  33 │    77      1


In [26]:
# filter out every group that has the same entry in Subject
naive_filtered_groups_only_sev_subj = filter(g -> length(unique(g.Subject)) > 1, naive_subj_filtered_grouped)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String15,String15,String7,String15,String31,Int64,String15,String15,String,String
1,AGCTCTGGGAGAAGAGCCCCAGCCCCAGAATTCCCAGGAGTTTCCATTCGGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGGTTTTCCTTGTTGCTATTATAAAAGGTGTCCAGTGTCAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAGTGACTACTACATGAGCTGGATCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTTCATACATTAGTAGTAGTAGTAGTTACACAAACTACGCAGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACGGCTGTGTATTACTGTGCGAGAGATCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-11*06,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAGTGACTACTACATGAGCTGGATCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTTCATACATTAGTAGTAGTAGTAGTTACACAAACTACGCAGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACGGCTGTGTATTACTGTGCGAGAGATCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLEWVSYISSSSSYTNYADSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARDLDYWGQGTLVTVSS,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLEWVSYISSSSSYTNYADSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARXXDYWGQGTLVTVSS,ARDLDY,TGGGGGCTGGGGTCTCAGGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTATTCCTCACCCTCCTCACTCAGGGCACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCTGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGACGTTGGTGGTTATAACTATGTCTCCTGGTACCAACAACACCCAGGCAAAGCCCCCAAACTCATGATTTATGATGTCAGTAATCGGCCCTCAGGGGTTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGACGAGGCTGATTATTACTGCAGCTCATATACAAGCAGCAGGGGATATGTCTTCGGAACTGGGACCAAGGTCACCGTCCTAGGTCAGCCCAAGGCCAACCCCACTGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTCCAAGCCAACAAGGCCACACTAGTGTGTCTGATCAGTGACTTCTACCCGGGAGCTGTGACAGTGGCCTGGAAGGCAGATGGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCAAACCCTCCAAACAGAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV2-14*03,CAGTCTGCCCTGACTCAGCCTGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGACGTTGGTGGTTATAACTATGTCTCCTGGTACCAACAACACCCAGGCAAAGCCCCCAAACTCATGATTTATGATGTCAGTAATCGGCCCTCAGGGGTTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGACGAGGCTGATTATTACTGCAGCTCATATACAAGCAGCAGGGGATATGTCTTCGGAACTGGGACCAAGGTCACCGTCCTAG,QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSRGYVFGTGTKVTVL,QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSXXYVFGTGTKVTVL,SSYTSSRGYV,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLEWVSYISSSSSYTNYADSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARDLDYWGQGTLVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSRGYVFGTGTKVTVL,Naive-B-Cells,CMV,human,Donor-4,"Jaffe et al., 2022",50,IGHV3-11,IGLV2-14,IGHV3-11,IGLV2-14
2,AGCTCTGGGAGAAGAGCCCCAGCCCCAGAATTCCCAGGAGTTTCCATTCGGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGGTTTTCCTTGTTGCTATTATAAAAGGTGTCCAGTGTCAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAGTGACTACTACATGAGCTGGATCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTTCATACATTAGTAGTAGTGGTAGTACCATATACTACGCAGACTCTGTGAAGGGCCGATTCACCATCTCCAGGGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACGGCCGTGTATTACTGTGCGAGAGATCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-11*01,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAGTGACTACTACATG

In [27]:
# alternative: Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases = 0
total_groups = length(naive_filtered_groups_only_sev_subj)

for group in naive_filtered_groups_only_sev_subj
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes = unique(group.general_v_gene_light)
    if length(unique_genes) == 1
        true_cases += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true = (true_cases / total_groups) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true%")
fraction_true

Percentage of groups where all entries have the same general_v_gene_light: 13.83399209486166%


13.83399209486166

# Unsorted B Cells (as "control")

In [28]:
uns_subj_df = filter(row -> row.BType == "Unsorted-B-Cells", subjects_df);

In [29]:
uns_subj_df = unique(uns_subj_df)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String,String,String7,String15,String31,String15
1,GATTTCCTTAAATTCAGGGTCCAGCTCACATGGGAAATACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV4-59*01,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAG,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXYSSXXXXXXYFDYWGQGTLVTVSS,ARDVGPYNSISPGRYYFDY,GGGGTCACAAGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTGCTCCTCACCCTCCTCACTCAGGACACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAGGTCAGCCCAAGGCCAACCCCACTGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTCCAAGCCAACAAGGCCACACTAGTGTGTCTGATCAGTGACTTCTACCCGGGAGCTGTGACAGTGGCCTGGAAGGCAGATGGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCAAACCCTCCAAACAGAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV2-23*02,CAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAG,QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,QSALTQPASVSGSPGQSITISCTGTSSDVGSYNLVSWYQQHPGKAPKLMIYEVSKRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSXXXYVFGTGTKVTVL,CSYAGSRILYV,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,Unsorted-B-Cells,None,human,390c,"James et al, 2020",65 to 70
2,ATACTTTCTGAGAGCCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGAGGGGTCGTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCATCTGCAATGTCACTGGTTTCTCCATCAGTGGTTACTTCTGGAGCTGGGTCCGGCAGCCCCCAGGGAAGGGACTGGAATGGATTGCCTCTATGACTTACAGTGGCACCACCAACTACAACCCCTCCCTCCAGGGCCGAGTCACTATGTCACTTTCCATGTCCAAGGACCAGGTCTCCCTGAAACTGAGCTCTGCGACCGCTGCGGACACGGCCGTCTATTACTGTGCGAGAATGGCGCGAGATGGCTACGTTTTGCGGGACTGGTACTTCGATCTCTGGGGCCAAGGCACCCTGGTCACTGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCC

In [30]:

# Extract up to the first two segments (e.g., "IGKV2-30" from "IGKV2-30*01")
uns_subj_df[!, :general_v_gene_heavy] = replace.(uns_subj_df.v_call_heavy, r"(^[^*]+?)(?:\*.*)?$" => s"\1");
uns_subj_df[!, :general_v_gene_light] = replace.(uns_subj_df.v_call_light, r"(^[^*]+?)(?:\*.*)?$" => s"\1");

unique_btypes = unique(uns_subj_df.BType)

println("unique BTypes: ", unique_btypes)


unique BTypes: ["Unsorted-B-Cells"]


In [31]:
# Filter and print entries with "D" in their name for each column
println("Entries with 'D' in general_v_gene_heavy:")
println(filter(row -> occursin("D", row[:general_v_gene_heavy]), uns_subj_df)[:, :general_v_gene_heavy])

println("Entries with 'D' in general_v_gene_light:")
println(filter(row -> occursin("D", row[:general_v_gene_light]), uns_subj_df)[:, :general_v_gene_light])


Entries with 'D' in general_v_gene_heavy:
["IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-43D", "IGHV3-4

In [32]:

# Add the new columns
uns_subj_df[!, :general_v_gene_heavy_no_para] = replace.(uns_subj_df.general_v_gene_heavy, r"D" => "")
uns_subj_df[!, :general_v_gene_light_no_para] = replace.(uns_subj_df.general_v_gene_light, r"D" => "")

# Display the modified DataFrame
println(first(uns_subj_df, 10))  # Print the first 10 rows for verification

10×25 DataFrame
 Row │ sequence_heavy                     locus_heavy  v_call_heavy   sequence_alignment_heavy           sequence_alignment_aa_heavy        germline_alignment_aa_heavy        cdr3_aa_heavy           sequence_light                     locus_light  v_call_light  sequence_alignment_light           sequence_alignment_aa_light        germline_alignment_aa_light        cdr3_aa_light  sequence_alignment_heavy_sep_light  BType             Disease  Species  Subject   Author             Age       general_v_gene_heavy  general_v_gene_light  general_v_gene_heavy_no_para  general_v_gene_light_no_para 
     │ String                             String1      String15       String                             String                             String                             String                  String                             String1      String15      String                             String                             String                             String31       String 

In [33]:
# Filter and print entries with "D" in their name for each column
println("Entries with 'D' in general_v_gene_heavy_no_para:")
println(filter(row -> occursin("D", row[:general_v_gene_heavy_no_para]), uns_subj_df)[:, :general_v_gene_heavy_no_para])

println("Entries with 'D' in general_v_gene_light_no_para:")
filter(row -> occursin("D", row[:general_v_gene_light_no_para]), uns_subj_df)[:, :general_v_gene_light_no_para]


Entries with 'D' in general_v_gene_heavy_no_para:
String[]
Entries with 'D' in general_v_gene_light_no_para:


String[]

In [34]:
uns_subj_df_counts = countmap(uns_subj_df.Subject)

Dict{String15, Int64} with 30 entries:
  "Patient-6"    => 1605
  "390c"         => 765
  "Patient-15"   => 3849
  "Patient-14"   => 4232
  "Patient-5"    => 3574
  "Subject-BCP4" => 425
  "Donor-45"     => 4056
  "Subject-BCP5" => 2935
  "Patient-2"    => 4162
  "None"         => 107411
  "Patient-1"    => 3340
  "Patient-12"   => 3314
  "Donor-2"      => 110051
  "Subject-BCP9" => 2976
  "Patient-7"    => 1584
  "Patient-3"    => 1812
  "Donor-3"      => 142088
  "Subject-BCP8" => 2887
  "Patient-13"   => 3482
  ⋮              => ⋮

In [35]:
# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
uns_subj_df_grouped = groupby(uns_subj_df, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
uns_subj_df_grouped_filtered = filter(g -> nrow(g) > 1, uns_subj_df_grouped)


Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String,String,String7,String15,String31,String15,String,String,String,String
1,GATTTCCTTAAATTCAGGGTCCAGCTCACATGGGAAATACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV4-59*01,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAG,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXYSSXXXXXXYFDYWGQGTLVTVSS,ARDVGPYNSISPGRYYFDY,GGGGTCACAAGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTGCTCCTCACCCTCCTCACTCAGGACACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAGGTCAGCCCAAGGCCAACCCCACTGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTCCAAGCCAACAAGGCCACACTAGTGTGTCTGATCAGTGACTTCTACCCGGGAGCTGTGACAGTGGCCTGGAAGGCAGATGGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCAAACCCTCCAAACAGAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV2-23*02,CAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAG,QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,QSALTQPASVSGSPGQSITISCTGTSSDVGSYNLVSWYQQHPGKAPKLMIYEVSKRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSXXXYVFGTGTKVTVL,CSYAGSRILYV,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,Unsorted-B-Cells,None,human,390c,"James et al, 2020",65 to 70,IGHV4-59,IGLV2-23,IGHV4-59,IGLV2-23
2,ATACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCAC

In [36]:
group_sizes_uns_subj = [nrow(group) for group in uns_subj_df_grouped_filtered]

size_counts_uns_subj = combine(groupby(DataFrame(size = group_sizes_uns_subj), :size), nrow => :count)

println(size_counts_uns_subj)

33×2 DataFrame
 Row │ size   count 
     │ Int64  Int64 
─────┼──────────────
   1 │     2  16493
   2 │     3   3695
   3 │     4   1741
   4 │     5    592
   5 │     6    301
   6 │     7    185
   7 │     8    128
   8 │     9     77
   9 │    10     51
  10 │    11     46
  11 │    12     28
  12 │    13     22
  13 │    14     15
  14 │    15      6
  15 │    16      4
  16 │    17      5
  17 │    18     11
  18 │    19      4
  19 │    20      2
  20 │    21      1
  21 │    22      5
  22 │    23      3
  23 │    24      1
  24 │    25      1
  25 │    26      2
  26 │    27      2
  27 │    28      2
  28 │    31      3
  29 │    35      1
  30 │    41      3
  31 │    42      1
  32 │   104      1
  33 │   140      1


In [37]:
# filter out every group that has the same entry in Subject
uns_filtered_groups_only_sev_subj = filter(g -> length(unique(g.Subject)) > 1, uns_subj_df_grouped_filtered)

Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String,String,String7,String15,String31,String15,String,String,String,String
1,AGGTCTCAGAGAGGAGCCTTAGCCCTGGACTCCAAGGCCTTTCCACTTGGTGATCAGCACTGAGCACAGAGGACTCACCATGGGATTGGGGCTGAGCTGGGTTTTCCTTGTTGCTCTTTTAGAAGTTGTCCAGTGTGAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCTGGGGAGTCCCTGAGACTCTCCTGTGAAGCCTCTGGAATCACGTTCAGTAGCTATTGGATGAGTTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTGGCCAACATAAAAAAAGATGGAAGTGAGACATGGTATGTGGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAACCTGAGAGTCGAGGACGCGGCTGTGTATTACTGTGGGAGAGGGAGTGGCTGGTTACAGGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,IGHV3-7*01,GAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCTGGGGAGTCCCTGAGACTCTCCTGTGAAGCCTCTGGAATCACGTTCAGTAGCTATTGGATGAGTTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTGGCCAACATAAAAAAAGATGGAAGTGAGACATGGTATGTGGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAACCTGAGAGTCGAGGACGCGGCTGTGTATTACTGTGGGAGAGGGAGTGGCTGGTTACAGGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,EVQLVESGGGLVQPGESLRLSCEASGITFSSYWMSWVRQAPGKGLEWVANIKKDGSETWYVDSVKGRFTISRDNAKNSLYLQMNNLRVEDAAVYYCGRGSGWLQDYWGQGTLVTVSS,EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMSWVRQAPGKGLEWVANIKQDGSEKYYVDSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARXSGWXXDYWGQGTLVTVSS,GRGSGWLQDY,GAGCTACAACAGGCAGGCAGGGGCAGCAAGATGGTGTTGCAGACCCAGGTCTTCATTTCTCTGTTGCTCTGGATCTCTGGTGCCTACGGGGACATCGTGATGACCCAGTCTCCAGACTCCCTGGCTGTGTCTCTGGGCGAGAGGGCCACCGTCAACTGCAAGTCCAGCCAGAGTGTTTTCTCTAGCTCCAGTAATAAGAACTTCCTAGCTTGGTTCCAGAAGAAACCAGGGCAGCCTCCTAAGTTGCTAATTTCCTGGGCATCTACCCGGGAATCCGGGGTCCCTGACCGATTCAGTGGCAGCGGGTCTGGGACAGATTTCACTCTCACCATCAGCAGCCTGCAGGCTGAAGATGTGGCAGTTTATTACTGTCAGCAATATCATAGTGGTCCGCTCACTTTCGGCGGAGGGACCAAGGTGGAGATCAAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,K,IGKV4-1*01,GACATCGTGATGACCCAGTCTCCAGACTCCCTGGCTGTGTCTCTGGGCGAGAGGGCCACCGTCAACTGCAAGTCCAGCCAGAGTGTTTTCTCTAGCTCCAGTAATAAGAACTTCCTAGCTTGGTTCCAGAAGAAACCAGGGCAGCCTCCTAAGTTGCTAATTTCCTGGGCATCTACCCGGGAATCCGGGGTCCCTGACCGATTCAGTGGCAGCGGGTCTGGGACAGATTTCACTCTCACCATCAGCAGCCTGCAGGCTGAAGATGTGGCAGTTTATTACTGTCAGCAATATCATAGTGGTCCGCTCACTTTCGGCGGAGGGACCAAGGTGGAGATCAAAC,DIVMTQSPDSLAVSLGERATVNCKSSQSVFSSSSNKNFLAWFQKKPGQPPKLLISWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYHSGPLTFGGGTKVEIK,DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPLTFGGGTKVEIK,QQYHSGPLT,EVQLVESGGGLVQPGESLRLSCEASGITFSSYWMSWVRQAPGKGLEWVANIKKDGSETWYVDSVKGRFTISRDNAKNSLYLQMNNLRVEDAAVYYCGRGSGWLQDYWGQGTLVTVSS[SEP]DIVMTQSPDSLAVSLGERATVNCKSSQSVFSSSSNKNFLAWFQKKPGQPPKLLISWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYHSGPLTFGGGTKVEIK,Unsorted-B-Cells,Multiple-sclerosis,human,None,"Ramesh et al, 2020",37.0,IGHV3-7,IGKV4-1,IGHV3-7,IGKV4-1
2,AGGTCTCAGAGAGGAGCCTTAGCCCTGGACTCCAAGGCCTTTCCACTTGGTGATCAGCACTGAGCACAGAGGACTCACCATGGAATTGGGGCTGACCTGGGTTTTCCTTGTTGCTATTTTAGAAGGTGTCCAGTGTGAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCGGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGAGTCCGCCTCAGTAACTATTGGATGAGCTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTGGCCAACATGAAGAAAGATGGAAGTGAGAAATGGTATGTGGACTCTGTGAAGGGCCGATTCACCATTTCCAGAGACAACGCCGAGAACTCACTGTTTCTGCAAATGGACAAACTGAGAGACGACGACACGGCTGTGTATTACTGCGGGAGGGGCAGTGGCTGGCTACAAGATTACTGGGGCCAGGGAATATCGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCTGCAGCACCCAGCCAGATGGGAACGTGGTCATCGCCTGCCTGGTCCAGGGCTTCTTCCCCCAGGAGC

In [38]:
# alternative: Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases_uns = 0
total_groups_uns = length(uns_filtered_groups_only_sev_subj)

for group in uns_filtered_groups_only_sev_subj
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes_uns = unique(group.general_v_gene_light_no_para)
    if length(unique_genes_uns) == 1
        true_cases_uns += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true_uns = (true_cases_uns / total_groups_uns) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true_uns%")
fraction_true_uns

Percentage of groups where all entries have the same general_v_gene_light: 64.45672191528546%


64.45672191528546

# unsorted b cells classified to naive and memory by classifier

In [39]:
#class_unsorted_b_cells_path = "/ibmm_data2/oas_database/paired_lea_tmp/paired_model/BERT2GPT/naive_memory_classification/test_results/unsorted_test_results_all_metrics_OAS_paired_classifier_batch_64_epochs_10_lr_1e-06_group_size_421255-2_unlabelled.csv"

class_unsorted_b_cells_path = "/ibmm_data/rodelc/HEAVY_TO_LIGHT/NAIVE_MEMORY_CLS/models/heavy_bs64_lr3e-6_wd0.01_do0.1/BEST_MODEL_checkpoint_epoch_200.pt/heavy_bs64_lr3e-6_wd0.01_do0.1/coherence_coherence_heavy_predictions.csv"

"/ibmm_data/rodelc/HEAVY_TO_LIGHT/NAIVE_MEMORY_CLS/models/heavy_bs64_lr3e-6_wd0.01_do0.1/BEST_MODEL_checkpoint_epoch_200.pt/heavy_bs64_lr3e-6_wd0.01_do0.1/coherence_coherence_heavy_predictions.csv"

In [40]:
class_unsorted_b_cells = CSV.read(class_unsorted_b_cells_path, DataFrame);


In [41]:
# divide by memory and naive (1 in predicted class for memory and 0 for naive)
class_unsorted_b_cells[!, :predicted_btype] = ifelse.(class_unsorted_b_cells.predicted_heavy_label .== 1, "Memory-B-Cells", "Naive-B-Cells");


In [42]:
class_unsorted_b_cells

Row,sequence_alignment_aa_heavy,v_call_heavy,cdr3_aa_heavy,sequence_alignment_aa_light,locus_light,v_call_light,sequence_alignment_heavy_sep_light,BType,label,predicted_heavy_label,predicted_btype
,String,String15,String,String,String1,String15,String,String31,Int64,Int64,String
1,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS,IGHV4-59*01,ARDVGPYNSISPGRYYFDY,QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,L,IGLV2-23*02,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,Unsorted-B-Cells,0,1,Memory-B-Cells
2,QVQLQESGPGLVKPSETLSLICNVTGFSISGYFWSWVRQPPGKGLEWIASMTYSGTTNYNPSLQGRVTMSLSMSKDQVSLKLSSATAADTAVYYCARMARDGYVLRDWYFDLWGQGTLVTVSS,IGHV4-59*01,ARMARDGYVLRDWYFDL,EIVLTQSPGTLSLSTGERATLSCRAGQTVDGNSLAWYQHKPGQAPRLLIFRASRRAADIPDRFTGSGSGTDFTLTISRLEVEDFAVYYCQQYGASPKTFGQGTKVE,K,IGKV3-20*01,QVQLQESGPGLVKPSETLSLICNVTGFSISGYFWSWVRQPPGKGLEWIASMTYSGTTNYNPSLQGRVTMSLSMSKDQVSLKLSSATAADTAVYYCARMARDGYVLRDWYFDLWGQGTLVTVSS[SEP]EIVLTQSPGTLSLSTGERATLSCRAGQTVDGNSLAWYQHKPGQAPRLLIFRASRRAADIPDRFTGSGSGTDFTLTISRLEVEDFAVYYCQQYGASPKTFGQGTKVE,Unsorted-B-Cells,0,1,Memory-B-Cells
3,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYHWSWIRQPPGKGLEWIGYMYYSGSTNYNPSLKSRVTISVDTSKTQFSLKLSSVTTADTAVYYCARGRLIWSADYTGGDYFDPWGQGILVTVSS,IGHV4-59*01,ARGRLIWSADYTGGDYFDP,QSALTQPASVSGSPGQSITISCTGSSSDVGSYNLVSWYQQHPGKAPKLMIYEVSKRPSGVSNRFSGSKSGNTASLTISGLQAEDEAQYYCCSYGGRNFHVLFGGGTELTVL,L,IGLV2-23*02,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYHWSWIRQPPGKGLEWIGYMYYSGSTNYNPSLKSRVTISVDTSKTQFSLKLSSVTTADTAVYYCARGRLIWSADYTGGDYFDPWGQGILVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGSSSDVGSYNLVSWYQQHPGKAPKLMIYEVSKRPSGVSNRFSGSKSGNTASLTISGLQAEDEAQYYCCSYGGRNFHVLFGGGTELTVL,Unsorted-B-Cells,0,1,Memory-B-Cells
4,QVQLQESGPGLVKPSETLSLTCNVSGYSISSGYYWGWIRQPPGKGLEWIGIIYQNGHSFYNPSLKSRAALSVAASKNQFSLNLRSVTAADTAVYFCARVASNAPTDWGQGTLVTVSS,IGHV4-38-2*02,ARVASNAPTD,QSALTQPPSASGSLGQSVTISCTGSSSDVGGYAYVSWYQQHPGKAPKVVIYEVTKRPSGVPERFSGSKSGNTASLTVSGLQAEDEADYYCISYAGANKLGVFGGGTKLTVL,L,IGLV2-8*01,QVQLQESGPGLVKPSETLSLTCNVSGYSISSGYYWGWIRQPPGKGLEWIGIIYQNGHSFYNPSLKSRAALSVAASKNQFSLNLRSVTAADTAVYFCARVASNAPTDWGQGTLVTVSS[SEP]QSALTQPPSASGSLGQSVTISCTGSSSDVGGYAYVSWYQQHPGKAPKVVIYEVTKRPSGVPERFSGSKSGNTASLTVSGLQAEDEADYYCISYAGANKLGVFGGGTKLTVL,Unsorted-B-Cells,0,1,Memory-B-Cells
5,QPQLQESGPRLVKPSETLSLTCSVSGGSITNDNYYWVWIRQPPGKGLDWVGSINYSGRTYYNPSLKSRLTMSVDTSKNQFSLKLTSVTAADTAIYYCARLFDPFVNDYSPGTGYGWLDPWGQGTPVTVSA,IGHV4-39*01,ARLFDPFVNDYSPGTGYGWLDP,DIQLTQSPSSLSASVGDRVTITCQATQDIRKSLNWYQQKPGKAPKLLINDASNLQTGVPSRFSGSGSGTDFSFTINTLQPEDIATYFCQQYRSLPLTFGGGSNVEI,K,IGKV1-33*01,QPQLQESGPRLVKPSETLSLTCSVSGGSITNDNYYWVWIRQPPGKGLDWVGSINYSGRTYYNPSLKSRLTMSVDTSKNQFSLKLTSVTAADTAIYYCARLFDPFVNDYSPGTGYGWLDPWGQGTPVTVSA[SEP]DIQLTQSPSSLSASVGDRVTITCQATQDIRKSLNWYQQKPGKAPKLLINDASNLQTGVPSRFSGSGSGTDFSFTINTLQPEDIATYFCQQYRSLPLTFGGGSNVEI,Unsorted-B-Cells,0,1,Memory-B-Cells
6,QVQLQVSGPGLVKPSETLSLTCSVSNYSIGSGYYWGWVRQPPGRGLEWIGSIFRNGNTYYNPSLQSRVTISVETSKNHFSLRLSPVTAADTAVYYCARHNRYNQRNPFDLWGQGTLVTVSS,IGHV4-38-2*02,ARHNRYNQRNPFDL,DIVMTQSPDSLAVSLGERATINCKSSQSVLDNSFNKNYLAWYQQKPGLPPKLLIYWAFTRESGVPDRFSGSGSGTDFTLTINSLQAEDVAIYYCQQYYTYPFTFGPGTKVDIK,K,IGKV4-1*01,QVQLQVSGPGLVKPSETLSLTCSVSNYSIGSGYYWGWVRQPPGRGLEWIGSIFRNGNTYYNPSLQSRVTISVETSKNHFSLRLSPVTAADTAVYYCARHNRYNQRNPFDLWGQGTLVTVSS[SEP]DIVMTQSPDSLAVSLGERATINCKSSQSVLDNSFNKNYLAWYQQKPGLPPKLLIYWAFTRESGVPDRFSGSGSGTDFTLTINSLQAEDVAIYYCQQYYTYPFTFGPGTKVDIK,Unsorted-B-Cells,0,1,Memory-B-Cells
7,QVQLQESGPGLVKSSETLSLTCTVSGVPISSSSYYWAWIRQPPGKGLEWIGSIYYSGSSFYHPSLGSRVTISMDKSKNVFSLKVHSLTAADTAVYFCARTALITYSYGEGRAFFDYWGQGRLVTVSS,IGHV4-39*07,ARTALITYSYGEGRAFFDY,QSALTQPASVSGSPGQSITISCSGTSSDVGNYNLISWYQQHPGKAPKVMIYEVSQRPSGVSSRFSGSKSGNTASLTISGLQPEDEADYYCSSYAGRYTFVFGAGTRVTVL,L,IGLV2-23*02,QVQLQESGPGLVKSSETLSLTCTVSGVPISSSSYYWAWIRQPPGKGLEWIGSIYYSGSSFYHPSLGSRVTISMDKSKNVFSLKVHSLTAADTAV

In [ ]:
# # Extract up to the first two segments (e.g., "IGKV2-30" from "IGKV2-30*01")
# class_unsorted_b_cells[!, :general_v_gene_heavy] = replace.(class_unsorted_b_cells.v_call_heavy, r"(^[^*]+?)(?:\*.*)?$" => s"\1");
# class_unsorted_b_cells[!, :general_v_gene_light] = replace.(class_unsorted_b_cells.v_call_light, r"(^[^*]+?)(?:\*.*)?$" => s"\1");


In [ ]:
# # Add the new columns
# class_unsorted_b_cells[!, :general_v_gene_heavy_no_para] = replace.(class_unsorted_b_cells.general_v_gene_heavy, r"D" => "")
# class_unsorted_b_cells[!, :general_v_gene_light_no_para] = replace.(class_unsorted_b_cells.general_v_gene_light, r"D" => "")


634701-element Vector{String}:
 "IGLV2-23"
 "IGKV3-20"
 "IGLV2-23"
 "IGLV2-8"
 "IGKV1-33"
 "IGKV4-1"
 "IGLV2-23"
 "IGKV1-12"
 "IGLV2-14"
 "IGLV3-1"
 ⋮
 "IGLV1-44"
 "IGKV4-1"
 "IGKV1-5"
 "IGKV3-11"
 "IGLV3-27"
 "IGKV1-39"
 "IGKV1-39"
 "IGLV6-57"
 "IGLV7-43"

In [43]:
# merge class_unsorted_b_cells and uns_subj_df on sequence_alignment_aa_heavy and BType
merged_unsorted_b_cells = innerjoin(class_unsorted_b_cells, uns_subj_df, on = [:sequence_alignment_aa_heavy, :BType, :v_call_heavy, :cdr3_aa_heavy, :locus_light, :v_call_light, :sequence_alignment_aa_light, :sequence_alignment_heavy_sep_light])

Row,sequence_alignment_aa_heavy,v_call_heavy,cdr3_aa_heavy,sequence_alignment_aa_light,locus_light,v_call_light,sequence_alignment_heavy_sep_light,BType,label,predicted_heavy_label,predicted_btype,sequence_heavy,locus_heavy,sequence_alignment_heavy,germline_alignment_aa_heavy,sequence_light,sequence_alignment_light,germline_alignment_aa_light,cdr3_aa_light,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String15,String,String,String1,String15,String,String31,Int64,Int64,String,String,String1,String,String,String,String,String,String31,String,String7,String15,String31,String15,String,String,String,String
1,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS,IGHV4-59*01,ARDVGPYNSISPGRYYFDY,QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,L,IGLV2-23*02,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,Unsorted-B-Cells,0,1,Memory-B-Cells,GATTTCCTTAAATTCAGGGTCCAGCTCACATGGGAAATACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAG,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXYSSXXXXXXYFDYWGQGTLVTVSS,GGGGTCACAAGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTGCTCCTCACCCTCCTCACTCAGGACACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAGGTCAGCCCAAGGCCAACCCCACTGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTCCAAGCCAACAAGGCCACACTAGTGTGTCTGATCAGTGACTTCTACCCGGGAGCTGTGACAGTGGCCTGGAAGGCAGATGGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCAAACCCTCCAAACAGAGCAACAACAAGTACGCGGCCAGCAGCTA,CAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAG,QSALTQPASVSGSPGQSITISCTGTSSDVGSYNLVSWYQQHPGKAPKLMIYEVSKRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSXXXYVFGTGTKVTVL,CSYAGSRILYV,None,human,390c,"James et al, 2020",65 to 70,IGHV4-59,IGLV2-23,IGHV4-59,IGLV2-23
2,QVQLQESGPGLVKPSETLSLICNVTGFSISGYFWSWVRQPPGKGLEWIASMTYSGTTNYNPSLQGRVTMSLSMSKDQVSLKLSSATAADTAVYYCARMARDGYVLRDWYFDLWGQGTLVTVSS,IGHV4-59*01,ARMARDGYVLRDWYFDL,EIVLTQSPGTLSLSTGERATLSCRAGQTVDGNSLAWYQHKPGQAPRLLIFRASRRAADIPDRFTGSGSGTDFTLTISRLEVEDFAVYYCQQYGASPKTFGQGTKVE,K,IGKV3-20*01,QVQLQESGPGLVKPSETLSLICNVTGFSISGYFWSWVRQPPGKGLEWIASMTYSGTTNYNPSLQGRVTMSLSMSKDQVSLKLSSATAADTAVYYCARMAR

In [45]:
merged_unsorted_b_cells_unique = unique(merged_unsorted_b_cells, :sequence_alignment_heavy_sep_light)

Row,sequence_alignment_aa_heavy,v_call_heavy,cdr3_aa_heavy,sequence_alignment_aa_light,locus_light,v_call_light,sequence_alignment_heavy_sep_light,BType,label,predicted_heavy_label,predicted_btype,sequence_heavy,locus_heavy,sequence_alignment_heavy,germline_alignment_aa_heavy,sequence_light,sequence_alignment_light,germline_alignment_aa_light,cdr3_aa_light,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String15,String,String,String1,String15,String,String31,Int64,Int64,String,String,String1,String,String,String,String,String,String31,String,String7,String15,String31,String15,String,String,String,String
1,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS,IGHV4-59*01,ARDVGPYNSISPGRYYFDY,QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,L,IGLV2-23*02,QVQLQESGPGLVKPSETLSLTCTVSGGSISGFYWSWIRQSPGKGLEWIAYIYFSGSTNYNPSLKSRVTLSVDTSKNQFSLKLSSVTAADSAVYYCARDVGPYNSISPGRYYFDYWGPGTLVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGTSSDVGNYNLVSWYQHHPGKAPKLMIYEVSKRPSGISNRFSGSKSGNTASLTISGLQADDEADYYCCSYAGSRILYVFGSGTKVTVL,Unsorted-B-Cells,0,1,Memory-B-Cells,GATTTCCTTAAATTCAGGGTCCAGCTCACATGGGAAATACTTTCTGAGAGTCCTGGACCTCCTGTGCAAGAACATGAAACATCTGTGGTTCTTCCTTCTCCTGGTGGCAGCTCCCAGATGGGTCCTGTCCCAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGGAGACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATTAGTGGTTTTTATTGGAGCTGGATCCGGCAGTCCCCAGGGAAGGGACTGGAGTGGATTGCATATATCTATTTTAGTGGGAGCACCAACTACAACCCCTCCCTCAAGAGTCGAGTCACCTTATCAGTGGACACGTCCAAGAACCAGTTCTCCCTGAAACTGAGCTCTGTTACCGCTGCGGACTCGGCCGTTTATTACTGTGCGAGAGATGTCGGCCCGTATAACAGCATCTCCCCGGGGCGTTACTATTTTGACTACTGGGGCCCGGGAACCCTGGTCACCGTCTCCTCAG,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYYSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARXXXXYSSXXXXXXYFDYWGQGTLVTVSS,GGGGTCACAAGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTGCTCCTCACCCTCCTCACTCAGGACACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAGGTCAGCCCAAGGCCAACCCCACTGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTCCAAGCCAACAAGGCCACACTAGTGTGTCTGATCAGTGACTTCTACCCGGGAGCTGTGACAGTGGCCTGGAAGGCAGATGGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCAAACCCTCCAAACAGAGCAACAACAAGTACGCGGCCAGCAGCTA,CAGTCTGCCCTGACTCAGCCAGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGATGTTGGGAATTATAACCTTGTCTCCTGGTACCAACACCACCCAGGCAAAGCCCCCAAACTCATGATTTATGAGGTCAGTAAGCGGCCCTCAGGGATTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACAATCTCTGGGCTCCAGGCTGACGACGAGGCTGATTATTACTGCTGCTCATATGCAGGTAGTAGAATCCTTTATGTCTTCGGATCTGGGACCAAGGTCACCGTCCTAG,QSALTQPASVSGSPGQSITISCTGTSSDVGSYNLVSWYQQHPGKAPKLMIYEVSKRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSXXXYVFGTGTKVTVL,CSYAGSRILYV,None,human,390c,"James et al, 2020",65 to 70,IGHV4-59,IGLV2-23,IGHV4-59,IGLV2-23
2,QVQLQESGPGLVKPSETLSLICNVTGFSISGYFWSWVRQPPGKGLEWIASMTYSGTTNYNPSLQGRVTMSLSMSKDQVSLKLSSATAADTAVYYCARMARDGYVLRDWYFDLWGQGTLVTVSS,IGHV4-59*01,ARMARDGYVLRDWYFDL,EIVLTQSPGTLSLSTGERATLSCRAGQTVDGNSLAWYQHKPGQAPRLLIFRASRRAADIPDRFTGSGSGTDFTLTISRLEVEDFAVYYCQQYGASPKTFGQGTKVE,K,IGKV3-20*01,QVQLQESGPGLVKPSETLSLICNVTGFSISGYFWSWVRQPPGKGLEWIASMTYSGTTNYNPSLQGRVTMSLSMSKDQVSLKLSSATAADTAVYYCARMAR

In [46]:
# group by predicted_btype
grouped_by_predicted_btype = groupby(merged_unsorted_b_cells_unique, :predicted_btype)

# Extract the first and second groups into separate DataFrames:
df_pred_memory = DataFrame(grouped_by_predicted_btype[1])
df_pred_naive = DataFrame(grouped_by_predicted_btype[2])

print(size(df_pred_memory, 1), " ", size(df_pred_naive, 1))




266112 368589

# Memory classified from unsorted-B-Cells

In [47]:
# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
df_pred_memory_grouped = groupby(df_pred_memory, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
df_pred_memory_grouped_filt = filter(g -> nrow(g) > 1, df_pred_memory_grouped)

# filter out every group that has the same entry in Subject
df_pred_memory_grouped_filt_o1s = filter(g -> length(unique(g.Subject)) > 1, df_pred_memory_grouped_filt)



Row,sequence_alignment_aa_heavy,v_call_heavy,cdr3_aa_heavy,sequence_alignment_aa_light,locus_light,v_call_light,sequence_alignment_heavy_sep_light,BType,label,predicted_heavy_label,predicted_btype,sequence_heavy,locus_heavy,sequence_alignment_heavy,germline_alignment_aa_heavy,sequence_light,sequence_alignment_light,germline_alignment_aa_light,cdr3_aa_light,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String15,String,String,String1,String15,String,String31,Int64,Int64,String,String,String1,String,String,String,String,String,String31,String,String7,String15,String31,String15,String,String,String,String
1,EVQLVESGGGLVQPGESLRLSCEASGITFSSYWMSWVRQAPGKGLEWVANIKKDGSETWYVDSVKGRFTISRDNAKNSLYLQMNNLRVEDAAVYYCGRGSGWLQDYWGQGTLVTVSS,IGHV3-7*01,GRGSGWLQDY,DIVMTQSPDSLAVSLGERATVNCKSSQSVFSSSSNKNFLAWFQKKPGQPPKLLISWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYHSGPLTFGGGTKVEIK,K,IGKV4-1*01,EVQLVESGGGLVQPGESLRLSCEASGITFSSYWMSWVRQAPGKGLEWVANIKKDGSETWYVDSVKGRFTISRDNAKNSLYLQMNNLRVEDAAVYYCGRGSGWLQDYWGQGTLVTVSS[SEP]DIVMTQSPDSLAVSLGERATVNCKSSQSVFSSSSNKNFLAWFQKKPGQPPKLLISWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYHSGPLTFGGGTKVEIK,Unsorted-B-Cells,0,1,Memory-B-Cells,AGGTCTCAGAGAGGAGCCTTAGCCCTGGACTCCAAGGCCTTTCCACTTGGTGATCAGCACTGAGCACAGAGGACTCACCATGGGATTGGGGCTGAGCTGGGTTTTCCTTGTTGCTCTTTTAGAAGTTGTCCAGTGTGAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCTGGGGAGTCCCTGAGACTCTCCTGTGAAGCCTCTGGAATCACGTTCAGTAGCTATTGGATGAGTTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTGGCCAACATAAAAAAAGATGGAAGTGAGACATGGTATGTGGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAACCTGAGAGTCGAGGACGCGGCTGTGTATTACTGTGGGAGAGGGAGTGGCTGGTTACAGGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGCATCCCCGACCAGCCCCAAGGTCTTCCCGCTGAGCCTCGACAGCACCCCCCAAGATGGGAACGTGGTCGTCGCATGCCTGGTCCAGGGCTTCTTCCCCCAGGAGCCACTCAGTGTGACCTGGAGCGAAAGCGGACAGAACGTGACCGCCAGAAACTTCCC,H,GAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCCAGCCTGGGGAGTCCCTGAGACTCTCCTGTGAAGCCTCTGGAATCACGTTCAGTAGCTATTGGATGAGTTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTGGCCAACATAAAAAAAGATGGAAGTGAGACATGGTATGTGGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAACCTGAGAGTCGAGGACGCGGCTGTGTATTACTGTGGGAGAGGGAGTGGCTGGTTACAGGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,EVQLVESGGGLVQPGGSLRLSCAASGFTFSSYWMSWVRQAPGKGLEWVANIKQDGSEKYYVDSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARXSGWXXDYWGQGTLVTVSS,GAGCTACAACAGGCAGGCAGGGGCAGCAAGATGGTGTTGCAGACCCAGGTCTTCATTTCTCTGTTGCTCTGGATCTCTGGTGCCTACGGGGACATCGTGATGACCCAGTCTCCAGACTCCCTGGCTGTGTCTCTGGGCGAGAGGGCCACCGTCAACTGCAAGTCCAGCCAGAGTGTTTTCTCTAGCTCCAGTAATAAGAACTTCCTAGCTTGGTTCCAGAAGAAACCAGGGCAGCCTCCTAAGTTGCTAATTTCCTGGGCATCTACCCGGGAATCCGGGGTCCCTGACCGATTCAGTGGCAGCGGGTCTGGGACAGATTTCACTCTCACCATCAGCAGCCTGCAGGCTGAAGATGTGGCAGTTTATTACTGTCAGCAATATCATAGTGGTCCGCTCACTTTCGGCGGAGGGACCAAGGTGGAGATCAAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,GACATCGTGATGACCCAGTCTCCAGACTCCCTGGCTGTGTCTCTGGGCGAGAGGGCCACCGTCAACTGCAAGTCCAGCCAGAGTGTTTTCTCTAGCTCCAGTAATAAGAACTTCCTAGCTTGGTTCCAGAAGAAACCAGGGCAGCCTCCTAAGTTGCTAATTTCCTGGGCATCTACCCGGGAATCCGGGGTCCCTGACCGATTCAGTGGCAGCGGGTCTGGGACAGATTTCACTCTCACCATCAGCAGCCTGCAGGCTGAAGATGTGGCAGTTTATTACTGTCAGCAATATCATAGTGGTCCGCTCACTTTCGGCGGAGGGACCAAGGTGGAGATCAAAC,DIVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKNYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSTPLTFGGGTKVEIK,QQYHSGPLT,Multiple-sclerosis,human,None,"Ramesh et al, 2020",37.0,IGHV3-7,IGKV4-1,IGHV3-7,IGKV4-1
2,EVQLVESGGGLVQPGGSLRLSCAASGVRLSNYWMSWVRQAPGKGLEWVANMKKDGSEKWYVDSVKGRFTISRDNAENSLFLQMDKLRDDDTAVYYCGRGSGWLQDYWGQGISVTVSS,IGHV3-7*04,GRGSGWLQDY,DIVMTQSPDSLAVSLGERATVNCKSSQSILSSSNNKNYLAWFHKKPGQPPKLLISWASTRESGVPDRFSGSGSGTDFTLTINSLQAEDVAVYYCQQYYGGPLTFGGGTKVEIK,K,IGKV4-1*01,EVQLVESGGGLVQPGGSLRLSCAASGVRLSNYWMSWVRQAPGKGLEWVANMKKDGSEKWYVDSVKGRFTISRDNAENSLFLQMDKLRDDDTAVYYCGRGSGWLQDYWGQGISVTVSS[SEP]DIVMTQSPDSLAVSLGERATVNCKSSQSILSSSNNKNYLAWFHKKPGQPPKLLISWASTRESGVPDRFSGSGSGTDFTLTINSLQAEDVAVYYCQQYYGGPLTFGGGTKVEIK,Unsort

In [48]:
# alternative: Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases_uns = 0
total_groups_uns = length(df_pred_memory_grouped_filt_o1s)

for group in df_pred_memory_grouped_filt_o1s
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes_uns = unique(group.general_v_gene_light_no_para)
    if length(unique_genes_uns) == 1
        true_cases_uns += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true_uns = (true_cases_uns / total_groups_uns) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true_uns%")
fraction_true_uns

Percentage of groups where all entries have the same general_v_gene_light: 80.57142857142857%


80.57142857142857

# Naive-B-Cells classified from unsorted-B-Cells

In [49]:
# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
df_pred_naive_grouped = groupby(df_pred_naive, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
df_pred_naive_grouped_filt = filter(g -> nrow(g) > 1, df_pred_naive_grouped)

# filter out every group that has the same entry in Subject
df_pred_naive_grouped_filt_o1s = filter(g -> length(unique(g.Subject)) > 1, df_pred_naive_grouped_filt)


Row,sequence_alignment_aa_heavy,v_call_heavy,cdr3_aa_heavy,sequence_alignment_aa_light,locus_light,v_call_light,sequence_alignment_heavy_sep_light,BType,label,predicted_heavy_label,predicted_btype,sequence_heavy,locus_heavy,sequence_alignment_heavy,germline_alignment_aa_heavy,sequence_light,sequence_alignment_light,germline_alignment_aa_light,cdr3_aa_light,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para
,String,String15,String,String,String1,String15,String,String31,Int64,Int64,String,String,String1,String,String,String,String,String,String31,String,String7,String15,String31,String15,String,String,String,String
1,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLEWVSAISGSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKDLDYWGQGTLVTVSS,IGHV3-23*01,AKDLDY,DIQMTQSPSSLSASVGDRVTITCRASQGISNYLAWYQQKPGKVPKLLIYAASTLQSGVPSRFSGSGSGTDFTLTISSLQPEDVATYYCQKYNSAPWTFGQGTKVEIK,K,IGKV1-27*01,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLEWVSAISGSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKDLDYWGQGTLVTVSS[SEP]DIQMTQSPSSLSASVGDRVTITCRASQGISNYLAWYQQKPGKVPKLLIYAASTLQSGVPSRFSGSGSGTDFTLTISSLQPEDVATYYCQKYNSAPWTFGQGTKVEIK,Unsorted-B-Cells,0,0,Naive-B-Cells,AGCTCTGAGAGAGGAGCCCAGCCCTGGGATTTTCAGGTGTTTTCATTTGGTGATCAGGACTGAACAGAGAGAACTCACCATGGAGTTTGGGCTGAGCTGGCTTTTTCTTGTGGCTATTTTAAAAGGTGTCCAGTGTGAGGTGCAGCTGTTGGAGTCTGGGGGAGGCTTGGTACAGCCTGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTAGCAGCTATGCCATGAGCTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTCTCAGCTATTAGTGGTAGTGGTGGTAGCACATACTACGCAGACTCCGTGAAGGGCCGGTTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACGGCCGTATATTACTGTGCGAAAGATCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,GAGGTGCAGCTGTTGGAGTCTGGGGGAGGCTTGGTACAGCCTGGGGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTAGCAGCTATGCCATGAGCTGGGTCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTCTCAGCTATTAGTGGTAGTGGTGGTAGCACATACTACGCAGACTCCGTGAAGGGCCGGTTCACCATCTCCAGAGACAATTCCAAGAACACGCTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACGGCCGTATATTACTGTGCGAAAGATCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLEWVSAISGSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKXXDYWGQGTLVTVSS,TGGGAGAAGTCTCTCTCAGTCAGGACACAGCATGGACATGAGGGTCCCTGCTCAGCTCCTGGGACTCCTGCTGCTCTGGCTCCCAGATACCAGATGTGACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCGAGTCAGGGCATTAGCAATTATTTAGCCTGGTATCAGCAGAAACCAGGGAAAGTTCCTAAGCTCCTGATCTATGCTGCATCCACTTTGCAATCAGGGGTCCCATCTCGGTTCAGTGGCAGTGGATCTGGGACAGATTTCACTCTCACCATCAGCAGCCTGCAGCCTGAAGATGTTGCAACTTATTACTGTCAAAAGTATAACAGTGCCCCCTGGACGTTCGGCCAAGGGACCAAGGTGGAAATCAAACGAACTGTGGCTGCACCATCTGTCTTCATCTTCCCGCCATCTGATGAGCAGTTGAAATCTGGAACTGCCTCTGTTGTGTGCCTGCTGAATAACTTCTATCCCAGAGAGGCCAAAGTACAGTGGAAGGTGGATAACGC,GACATCCAGATGACCCAGTCTCCATCCTCCCTGTCTGCATCTGTAGGAGACAGAGTCACCATCACTTGCCGGGCGAGTCAGGGCATTAGCAATTATTTAGCCTGGTATCAGCAGAAACCAGGGAAAGTTCCTAAGCTCCTGATCTATGCTGCATCCACTTTGCAATCAGGGGTCCCATCTCGGTTCAGTGGCAGTGGATCTGGGACAGATTTCACTCTCACCATCAGCAGCCTGCAGCCTGAAGATGTTGCAACTTATTACTGTCAAAAGTATAACAGTGCCCCCTGGACGTTCGGCCAAGGGACCAAGGTGGAAATCAAAC,DIQMTQSPSSLSASVGDRVTITCRASQGISNYLAWYQQKPGKVPKLLIYAASTLQSGVPSRFSGSGSGTDFTLTISSLQPEDVATYYCQKYNSAPWTFGQGTKVEIK,QKYNSAPWT,Multiple-sclerosis,human,None,"Ramesh et al, 2020",37.0,IGHV3-23,IGKV1-27,IGHV3-23,IGKV1-27
2,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLEWVSAISGSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKDLDYWGQGTLVTVSS,IGHV3-23*01,AKDLDY,QSALTQPRSVSGSPGQSVTISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSKRPSGVPDRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSYTYVVFGGGTKLTVL,L,IGLV2-11*01,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSYAMSWVRQAPGKGLEWVSAISGSGGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKDLDYWGQGTLVTVSS[SEP]QSALTQPRSVSGSPGQSVTISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSKRPSGVPDRFSGSKSGNTASLTISGLQAEDEADYYCCSYAGSYTYVVFGGGTKLTVL,Unsorted-B-Cells,0,0,Naive-B-Cells,AGCTCTGAGAGAGGAGCCCAGCCCTGGGATTTTCAGGTGTTTTCATTTGGTGATCAGGACTGAACAGAGAGAACTCACCATGGAGTTTGGGCTGAGCTGGCTTTTTCTTGTGGCTATTTTAAAAGGTGTCCAGTGTGAGGTGCAGCTGTTGGAGTC

In [50]:
# alternative: Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases_uns = 0
total_groups_uns = length(df_pred_naive_grouped_filt_o1s)

for group in df_pred_naive_grouped_filt_o1s
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes_uns = unique(group.general_v_gene_light_no_para)
    if length(unique_genes_uns) == 1
        true_cases_uns += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true_uns = (true_cases_uns / total_groups_uns) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true_uns%")
fraction_true_uns

Percentage of groups where all entries have the same general_v_gene_light: 5.952380952380952%


5.952380952380952

# All data (memory)

In [51]:
joined_df_memory_all = vcat(mem_subj_df, df_pred_memory, cols = :union);

In [52]:
# remove duplicates in the column sequence_alignment_heavy_sep_light
joined_df_memory_all = unique(joined_df_memory_all, :sequence_alignment_heavy_sep_light)
# save the dataframe to a csv file
CSV.write("/ibmm_data2/oas_database/paired_lea_tmp/paired_model/BERT2BERT/data/BType_Memory-B-Cells_extra_cols_joined.csv", joined_df_memory_all, writeheader=true)

"/ibmm_data2/oas_database/paired_lea_tmp/paired_model/BERT2BERT/data/BType_Memory-B-Cells_extra_cols_joined.csv"

In [53]:
# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
joined_df_memory_all_grouped = groupby(joined_df_memory_all, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
joined_df_memory_all_grouped_filt = filter(g -> nrow(g) > 1, joined_df_memory_all_grouped)

# filter out every group that has the same entry in Subject
joined_df_memory_all_grouped_filt_o1s = filter(g -> length(unique(g.Subject)) > 1, joined_df_memory_all_grouped_filt)


Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para,label,predicted_heavy_label,predicted_btype
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String31,String,String7,String15,String31,String15,String,String,String,String,Int64?,Int64?,String?
1,GAGCTCTGGGAGAGGAGCCCAGCACTAGAAGTCGGCGGTGTTTCCATTCGGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGGTTTTCCTCGTTGCTCTTTTAAGAGGTGTCCAGTGTCAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTAGTAGCTTTGGCATGCACTGGGTCCGGCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGGCGGTCATATCATTTTCATTTATGGGAAGTTTTGAATACTATTCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAGCACGCTGTATCTGCAAATGGACAACCTGAGAGTTGAGGACACGGCTGTATATTACTGTGTGAAAGAATCGAATGCTTTTGATGTCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-30*18,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCGTGGTCCAGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTAGTAGCTTTGGCATGCACTGGGTCCGGCAGGCTCCAGGCAAGGGGCTGGAGTGGGTGGCGGTCATATCATTTTCATTTATGGGAAGTTTTGAATACTATTCAGACTCCGTGAAGGGCCGATTCACCATCTCCAGAGACAATTCCAAGAGCACGCTGTATCTGCAAATGGACAACCTGAGAGTTGAGGACACGGCTGTATATTACTGTGTGAAAGAATCGAATGCTTTTGATGTCTGGGGCCAAGGGACAATGGTCACCGTCTCTTCAG,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSFGMHWVRQAPGKGLEWVAVISFSFMGSFEYYSDSVKGRFTISRDNSKSTLYLQMDNLRVEDTAVYYCVKESNAFDVWGQGTMVTVSS,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAVISYDGSNKYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKXXXAFDVWGQGTMVTVSS,VKESNAFDV,GAAAACAGAGCTTCAGCAAGCATAGTGGGAATCTGCACCATGCCCTGGGCTCTGCTCCTCCTGACCCTCCTCACTCACTCTGCAGTGTCAGTGGTCCAGGCAGGGCTGACTCAGCCACCCTCGGTGTCCAAGGGCCTGAGACAGACCGCCACAGTCACCTGCACTGGGAACAGCAACAATGTTGGCGACCAAGGAGCCGCTTGGCTGCAGCAGCACCAGGGCCACCCTCCCAAACTCCTTTCCTACAGGAATAACAACCGGCCCTCAGGGATCTCAGAGAGATTATCTGCATCCAGGTCAGGAAACACAGCCTCCCTGACCATTACTGGACTCCAGCCTGAGGACGAGGCTGACTATTACTGCTCAGCATGGGACAGTAGACTCAATGTTTGGGTGTTCGGCGGAGGGACCAAGCTGACCGTCCTAGGTCAGCCCAAGGCTGCCCCCTCGGTCACTCTGTTCCCACCCTCCTCTGAGGAGCTTCAAGCCAACAAGGCCACACTGGTGTGTCTCATAAGTGACTTCTACCCGGGAGCCGTGACAGTGGCCTGGAAGGCAGATAGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCACACCCTCCAAACAAAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV10-54*01,CAGGCAGGGCTGACTCAGCCACCCTCGGTGTCCAAGGGCCTGAGACAGACCGCCACAGTCACCTGCACTGGGAACAGCAACAATGTTGGCGACCAAGGAGCCGCTTGGCTGCAGCAGCACCAGGGCCACCCTCCCAAACTCCTTTCCTACAGGAATAACAACCGGCCCTCAGGGATCTCAGAGAGATTATCTGCATCCAGGTCAGGAAACACAGCCTCCCTGACCATTACTGGACTCCAGCCTGAGGACGAGGCTGACTATTACTGCTCAGCATGGGACAGTAGACTCAATGTTTGGGTGTTCGGCGGAGGGACCAAGCTGACCGTCCTAG,QAGLTQPPSVSKGLRQTATVTCTGNSNNVGDQGAAWLQQHQGHPPKLLSYRNNNRPSGISERLSASRSGNTASLTITGLQPEDEADYYCSAWDSRLNVWVFGGGTKLTVL,QAGLTQPPSVSKGLRQTATLTCTGNSNNVGNQGAAWLQQHQGHPPKLLSYRNNNRPSGISERLSASRSGNTASLTITGLQPEDEADYYCSAWDSSLSXWVFGGGTKLTVL,SAWDSRLNVWV,QVQLVESGGGVVQPGRSLRLSCAASGFTFSSFGMHWVRQAPGKGLEWVAVISFSFMGSFEYYSDSVKGRFTISRDNSKSTLYLQMDNLRVEDTAVYYCVKESNAFDVWGQGTMVTVSS[SEP]QAGLTQPPSVSKGLRQTATVTCTGNSNNVGDQGAAWLQQHQGHPPKLLSYRNNNRPSGISERLSASRSGNTASLTITGLQPEDEADYYCSAWDSRLNVWVFGGGTKLTVL,Memory-B-Cells,None,human,Donor-1,"Phad et al., 2022",no,IGHV3-30,IGLV10-54,IGHV3-30,IGLV10-54,missing,missing,missing
2,GGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGGTTTTCCTCGTTGCTCTTTTAAGAGGTGTCCAATGTCAGGTGCAGCTGGCGGAGTCTGGGGGAGGCTTAGTCCAGCCTGGGAGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTAACTCTCAGTAATTATGTCACGCACTGGGTCCGCCAGGCTCCAGGCAAGGGACCGGAGTGGCTGGCAGATATGTCACAGGATGGAAGTATTAGAATGTATGCAGGCTCCGTGAGGGGCCGATTCACCATCTCTAGAGACAATTCCGAGAACACTCTCTATCTGCAAATGAACAGCCTGAGAGTTGAGGACACGGGTGTATATTACTGTGTGAAAGAATCCAATGCTTTTGATGTCTGGGGCCAAGGGACAATGGTCATCGTCTCTTCAGCCTCCACCAAGGGCCCATCGGTCTTCCCCCTGGCGCCCTGCTCCAGGAGCACCTCCGAGAGCACAGCGGCCCTGGGCTGCCTGGTCAAGGACTACTT

In [88]:
# alternative: Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases_uns = 0
total_groups_uns = length(joined_df_memory_all_grouped_filt_o1s)

for group in joined_df_memory_all_grouped_filt_o1s
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes_uns = unique(group.general_v_gene_light_no_para)
    if length(unique_genes_uns) == 1
        true_cases_uns += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true_uns = (true_cases_uns / total_groups_uns) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true_uns%")
fraction_true_uns

Percentage of groups where all entries have the same general_v_gene_light: 60.63363667003707%


60.63363667003707

# All data (naive)

In [54]:
joined_df_naive_all = vcat(naive_subj_df, df_pred_naive, cols = :union);

In [55]:
# Group by `general_v_gene_heavy` and `cdr3_aa_heavy`
joined_df_naive_all_grouped = groupby(joined_df_naive_all, [:general_v_gene_heavy_no_para, :cdr3_aa_heavy])

# Step 1: Filter out groups with only one row
joined_df_naive_all_grouped_filt = filter(g -> nrow(g) > 1, joined_df_naive_all_grouped)

# filter out every group that has the same entry in Subject
joined_df_naive_all_grouped_filt_o1s = filter(g -> length(unique(g.Subject)) > 1, joined_df_naive_all_grouped_filt)


Row,sequence_heavy,locus_heavy,v_call_heavy,sequence_alignment_heavy,sequence_alignment_aa_heavy,germline_alignment_aa_heavy,cdr3_aa_heavy,sequence_light,locus_light,v_call_light,sequence_alignment_light,sequence_alignment_aa_light,germline_alignment_aa_light,cdr3_aa_light,sequence_alignment_heavy_sep_light,BType,Disease,Species,Subject,Author,Age,general_v_gene_heavy,general_v_gene_light,general_v_gene_heavy_no_para,general_v_gene_light_no_para,label,predicted_heavy_label,predicted_btype
,String,String1,String15,String,String,String,String,String,String1,String15,String,String,String,String31,String,String31,String,String7,String15,String31,Any,String,String,String,String,Int64?,Int64?,String?
1,AGCTCTGGGAGAAGAGCCCCAGCCCCAGAATTCCCAGGAGTTTCCATTCGGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGGTTTTCCTTGTTGCTATTATAAAAGGTGTCCAGTGTCAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAGTGACTACTACATGAGCTGGATCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTTCATACATTAGTAGTAGTAGTAGTTACACAAACTACGCAGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACGGCTGTGTATTACTGTGCGAGAGATCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-11*06,CAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAGTGACTACTACATGAGCTGGATCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTTCATACATTAGTAGTAGTAGTAGTTACACAAACTACGCAGACTCTGTGAAGGGCCGATTCACCATCTCCAGAGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACGGCTGTGTATTACTGTGCGAGAGATCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAG,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLEWVSYISSSSSYTNYADSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARDLDYWGQGTLVTVSS,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLEWVSYISSSSSYTNYADSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARXXDYWGQGTLVTVSS,ARDLDY,TGGGGGCTGGGGTCTCAGGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTATTCCTCACCCTCCTCACTCAGGGCACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCTGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGACGTTGGTGGTTATAACTATGTCTCCTGGTACCAACAACACCCAGGCAAAGCCCCCAAACTCATGATTTATGATGTCAGTAATCGGCCCTCAGGGGTTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGACGAGGCTGATTATTACTGCAGCTCATATACAAGCAGCAGGGGATATGTCTTCGGAACTGGGACCAAGGTCACCGTCCTAGGTCAGCCCAAGGCCAACCCCACTGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTCCAAGCCAACAAGGCCACACTAGTGTGTCTGATCAGTGACTTCTACCCGGGAGCTGTGACAGTGGCCTGGAAGGCAGATGGCAGCCCCGTCAAGGCGGGAGTGGAGACCACCAAACCCTCCAAACAGAGCAACAACAAGTACGCGGCCAGCAGCTA,L,IGLV2-14*03,CAGTCTGCCCTGACTCAGCCTGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAGTGACGTTGGTGGTTATAACTATGTCTCCTGGTACCAACAACACCCAGGCAAAGCCCCCAAACTCATGATTTATGATGTCAGTAATCGGCCCTCAGGGGTTTCTAATCGCTTCTCTGGCTCCAAGTCTGGCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGACGAGGCTGATTATTACTGCAGCTCATATACAAGCAGCAGGGGATATGTCTTCGGAACTGGGACCAAGGTCACCGTCCTAG,QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSRGYVFGTGTKVTVL,QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSXXYVFGTGTKVTVL,SSYTSSRGYV,QVQLVESGGGLVKPGGSLRLSCAASGFTFSDYYMSWIRQAPGKGLEWVSYISSSSSYTNYADSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYCARDLDYWGQGTLVTVSS[SEP]QSALTQPASVSGSPGQSITISCTGTSSDVGGYNYVSWYQQHPGKAPKLMIYDVSNRPSGVSNRFSGSKSGNTASLTISGLQAEDEADYYCSSYTSSRGYVFGTGTKVTVL,Naive-B-Cells,CMV,human,Donor-4,"Jaffe et al., 2022",50,IGHV3-11,IGLV2-14,IGHV3-11,IGLV2-14,missing,missing,missing
2,AGCTCTGGGAGAAGAGCCCCAGCCCCAGAATTCCCAGGAGTTTCCATTCGGTGATCAGCACTGAACACAGAGGACTCACCATGGAGTTTGGGCTGAGCTGGGTTTTCCTTGTTGCTATTATAAAAGGTGTCCAGTGTCAGGTGCAGCTGGTGGAGTCTGGGGGAGGCTTGGTCAAGCCTGGAGGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTCAGTGACTACTACATGAGCTGGATCCGCCAGGCTCCAGGGAAGGGGCTGGAGTGGGTTTCATACATTAGTAGTAGTGGTAGTACCATATACTACGCAGACTCTGTGAAGGGCCGATTCACCATCTCCAGGGACAACGCCAAGAACTCACTGTATCTGCAAATGAACAGCCTGAGAGCCGAGGACACGGCCGTGTATTACTGTGCGAGAGATCTTGACTACTGGGGCCAGGGAACCCTGGTCACCGTCTCCTCAGGGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTGAGAATTCCCCGTCGGATACGAGCAGCGTG,H,IGHV3-11*01,CAGGTGCAGCTGGTGGAGTC

In [56]:
# alternative: Calculate the fraction of groups where all entries have the same `general_v_gene_light`
true_cases_uns = 0
total_groups_uns = length(joined_df_naive_all_grouped_filt_o1s)

for group in joined_df_naive_all_grouped_filt_o1s
    # Check if all entries in the group have the same `general_v_gene_light`
    unique_genes_uns = unique(group.general_v_gene_light_no_para)
    if length(unique_genes_uns) == 1
        true_cases_uns += 1  # Increment the count if all genes are identical
    end
end

# Step 3: Calculate the fraction of "true" cases
fraction_true_uns = (true_cases_uns / total_groups_uns) * 100

println("Percentage of groups where all entries have the same general_v_gene_light: $fraction_true_uns%")
fraction_true_uns

Percentage of groups where all entries have the same general_v_gene_light: 11.190053285968029%


11.190053285968029